### Helping Customers' Journey
#### Team 14: Carlos Moreno, Elizabeth Khan, Jagan Lakshmipathy, and Ziling Huang
###### W261_SP22_FINAL_PROJECT_TEAM14
---

#### I. Phase Summaries

The purpose of this project is to facilitate a consulting study on how to improve customer experience by predicting flight departure delays for an online booking agency client. We aim to use ML to predict delayed flights to improve the customers’ experience with travel booking agencies.
Thus, we conducted research on the latest trends in predicting flight delays which include methods such as network representation of flight delays (graph approaches), statistical analysis (regression analysis), and machine learning (classification, prediction, recommendation systems). For the purposes of our approach, we frame this as a binary classification (logistic regression) problem where the outcome variable is the prediction whether a flight is delayed. A delayed flight is any flight that is 15 minutes later than the scheduled time. We may also consider a prediction problem (i.e. linear regression) to predict the outcome variable of time of delay in minutes.  

The ML pipeline was implemented across the Four Phases as follows:

| Phase 1: Data Preparation                  | Phase 2: Feature Preparation                              |
| ------------------------------------------ | --------------------------------------------------------- |
| 1.Six Months Flight Data                   | 1.Identify Categorical vs. Numeric variables              |
| 2.Select Key Flight Features               | 2.Split Train (1 to 4 month) and Validation (5-6 months)    |
| 3.Clean rows with null values              | 3.Define indexers, one-hot encoders and imputers                      |
| 4.Binary Outcome of Delayed >=15 min       | 4.Define model\_matrix\_stages                            |
| 5.Leverage Blob and Delta Lakes            | Define Scaler                                             |

| Phase 3: Model Setup                       | Phase 4: Model Performance                                |
| ------------------------------------------ | --------------------------------------------------------- |
| 1.Define Logistic Regression Model         | 1.Get summary for model and visualize performance         |
| 2.Define pipeline using matrix stages      |                                                           |
| 3.Build a parameter grid for tuning (LR)  |                                                           |
| 4.Execute CrossValidator                   |                                                           |
| 5.Fit the model                            |                                                           |
| 6.Select best model                        |                                                           |

##### a. Phase 1: Exploratory Data Analysis (EDA) and Joins
We performed an exploratory data analysis (EDA) on the 6-month flights dataset and weather dataset.  The flights data contains information on flights, airport locations, timings, departures and arrivals and their associated delays and root causes. Around 50% of the variables had more than 50% of its data missing, a majority of which were related to flight diversions. Most of the departure delays occurred with aircraft and previous flight arriving late. Data cleaning needs to be done to remove cancelled flights records from the dataset because they do not fall into the scope of the study on delays. In cancelled flights, we observed duplicates where flights were cancelled and then relaunched to a different location.

The weather data includes information on weather metrics which can be used by airlines in determining safety conditions for flight takeoff and landing. An analysis showed that poor visibility, low-hanging clouds, and low temperatures seem to be prevalent in instances where departure delays occur versus those when they do not. The weather data contains fields such as WND and CIG which have metrics concatenated with codes and requiring parsing to retrieve valuable information. We found missing values represented by '9's and duplicate records based on reporting frequency. In a correlation analysis performed, we did not observe strong correlation between weather and departure delay. For the missing airport and weather-related fields, we will consider various imputation methods such as mean, rolling averages, and kNN to clean up the data based on our understanding of the data.  

Finally, we created a reference table (airport_weather_ref) that maps the airport (airport_id) to its closest weather station (station_id) and its local time zone database for converting departure times to UTC times.  The distance in kilometers is calculated using the Haversine distance formula to determine the shortest distance between two points. The reference table was created through left joins on the flights table by using the flight dataset’s unique airport_ids (origin and destination) as the left table and then joining the Stations, Airport Location, and Timezone tables. We then used left joins to join the flights table to the reference table to convert local times to UTC and get the closest weather station id and as a result were able to join in the weather table based on the date timestamp and station id. The resulting tables from the joins were checkpointed and stored in parquet format on the Blob storage. We chose to use parquet format due to data query efficiency and compact storage benefits.

##### b. Phase 2: Feature Engineering

In Phase 2, we explored Logistic Regression, Linear Regression, and Gradient 
Boosted tree models using the spark implementation of a Model Pipeline. The 
pipeline addresses categorical features by using one hot encoding and normalizes 
numerical features using the StandardScaler. We also investigated various feature 
selection and reduction techniques such as Principal Component Analysis and 
forward selection. Forward Selection may be a preferred technique over principal 
component analysis as the features transformed into principal components values 
are less interpretable by humans. 

 Additionally, we solidified the Joins process to combine the flights, stations, and 
weather datasets and how to impute missing weather values. If weather values were missing upon joining on the closest weather station, we would use the next closest weather station data within a 6-mile radius. Any remaining missing values were handled by the Last Observation Carried 
Forward approach as it is suitable for a real time model prediction and only requires past 
data. Finally for feature engineering we created several new features based on the 
airport graphs such as airport popularity page rank and 
origin delay pairs. We also created categorical features for the season, weekday or weekend, and delay 
block. Time-based features created include a rolling prior ninety-day delay average and time-of-day. Our next steps will be looking into cross validation and how to deal with 
imbalanced data as we continue to evolve our ML pipeline.

A correlation analysis was performed to identify and potentially eliminate closely correlated variables such as departure information and arrival information to reduce multi-collinearity in our model. An analysis of the top and bottom 10 routes was made to identify routes where flight delay prediction is critical to customer satisfaction and revenue. A page rank was performed to identify the most important airports. Finally, we observed that there was some relationship between the month of the year and departure delays.  

Lastly, we completed a proof-of-concept logistic regression ML pipeline in which the features with missing values were imputed using a mean, categorical features were one-hot encoded, and numeric values were normalized using the standard deviation and mean of the training data. Since we are dealing with a time-series data set the months 1-4 were in the training set and months 5-6 were in the test set. We plan on refining the feature engineering, data imputation techniques, and optimizing the joins and overall ML pipeline in Phase 3.

##### c. Phase 3: Algorithm Exploration

During Phase 3, we worked on feature engineering for Centrality and Community Detection graph-based algorithm features using 
GraphFrame and from scratch (airport page rank weighted by prior delays and page rank weighted by prior number of 
flights). We started working on the multi-task machine learning algorithm by 
creating the logistic regression and linear regression models from scratch and 
testing the results against the Spark algorithms. We implemented a three-fold cross
validation for Logistic Regression, Gradient Boosted Trees, and Random Forest to 
find the optimal parameters. We explored how well our model performed compared 
to the baseline, and on seasonality dimensions (Season, Month, Time of Day). We 
saw that our models did not perform as well for the Fall season and performed the 
best for the Winter season. We did a gap analysis compared to the student 
leaderboard and found that while our model accuracy performs relatively well 
compared to others, we have an opportunity to improve the join performance using 
additional partitionby compared to what we currently have.

##### d. Phase 4: Algorithm (Pipeline) Optimization

In Phase IV, we first implemented Grid Search using 5% of the data due to the size of the data and time constraints.  Thus, we identified optimal parameters for Random Forest, Gradient Boosted Tree, Linear Support Vector Classifier and Logistic Regression. Random Forest was selected for further tuning as it has the best Grid Search Accuracy at 71.3%.  However, when Random Forest was trained and tested with the full data, the Accuracy and F2_score was around 64.8% and 52.5% respectively.

We then evaluated Random Forest performance across different variables - such as Season, Month, Time of the Day, Hour of the Day, Airport Group Ranking based on Number of Connections.  The breakdown that provided the most insights was analysis by Time of the Day (seven categories), Season (four categories), and Airport Ranking by Connections PageRank (four categories).  These analysis suggested that there is potential for implementing independent models by Time of the Day, Season and Airport Group.

We then proceeded to identify the most important variables leveraging Random Forest attribute importance.  Engineered features like "OD Delay Pair" and "Time of the Day" were the most important drivers of model performance with a weight of 23.1% and 22.1% respectively.  After investing significant effort in implementing PageRank from scratch and creating PageRank features, these features only reached an importance weight of 3.3%.  After this analysis, we selected the top 19 attributes (out of 33) that represented 95% of the attribute importance (weight). We also attempted to create features from Community Detection Algorithms but failed due to time and size constraints.

Using the reduced numbers of attributes, we explored several configurations implementing Grid Search with the whole data. Through this process we were able to improve the model performance achieving an `Accuracy`, `Recall` and `F2_score` of 65.7%, 63.6% and 52.2% respectively.

We implemented an Ensemble model where the predictions of the base models (Random Forest, Gradient Boosted Tree, Linear Support Vector Classifier) were used as input to a Logistic Regression model to predict delay (binary variable).  Unfortunately, this approach did not help to improve model performance.  

Finally, we implemented a Multi-task Algorithm - it combines binary classification task (logistic regression) and prediction task (linear regression) by adding their respective weighted loss functions together in a Multi-Task loss function. We trained it on 2.5% of the data and tested it on 2019 data and achieved an accuracy of 58.7%.

#### II. Question Formulation
Flight delay prediction is crucial in helping passengers avoid financial losses. Based on our estimates, the annual costs of domestic flight delays to passengers is around $5 billion (Airlines.org, 2018).

The main objective of the paper is to predict whether a flight will be delayed given a 2-hour time horizon. Departure delay is defined as a delay exceeding 15 minutes. We are interested in identifying significant drivers of delays and understanding how delays propagate through a network through exploratory data analysis and creating features to enhance our predictions. Much research effort in the past has been invested in this area to achieve better traffic management. The Bureau of Transport Statistics explains the main cause of delays as follows:

- Air Carrier: The cause of the cancellation or delay was due to circumstances within the airline's control (e.g. maintenance or crew problems, aircraft cleaning, baggage loading, fueling, etc.).
- Extreme Weather: Significant meteorological conditions (actual or forecasted) that, in the judgment of the carrier, delays or prevents the operation of a flight such as tornado, blizzard or hurricane.
- National Aviation System (NAS): Delays and cancellations attributable to the national aviation system that refer to a broad set of conditions, such as non-extreme weather conditions, airport operations, heavy traffic volume, and air traffic control.
- Late-arriving aircraft: A previous flight with same aircraft arrived late, causing the present flight to depart late.
- Security: Delays or cancellations caused by evacuation of a terminal or concourse, re-boarding of aircraft because of security breach, inoperative - screening equipment and/or long lines in excess of 29 minutes at screening areas.

Our hypothesis is that airport-specific factors, temporal factors and late-arriving aircraft has an impact on flight leaving on time. 
 
The stakeholders we serve through this study is a mid-size online booking agencies who have hired us as consultants for a program to improve customer loyalty and increase market share by offering a novel service where passengers are alerted about potential delays 2 hours before departure. Our goal is to leverage the power of machine learning to help customers reach their destination by alerting them of potential delays.

The booking agency is in a competitive environment with relatively low margins, and it is looking for ways to distinguish itself from competitors.  Research indicates that passengers would appreciate being alerted about potential delays so that they are able to make adjustments.  By offering this service, the booking agency is looking to increase loyalty among current customers which would lead to in an increase in repeat business.  In addition, the current customer increased loyalty may also increase customer referrals which may expand current customer base.  

Flight delay prediction is tackled from a few perspectives: (i) delay propagation and (ii) root delay (Carvhalo et al, 2017). Delay propagation refers to the chain reaction mechanism through which delays spread. Root delay refers to predicting when a new delay will occur impacting the flight network. Our paper attempts to tackle this mainly using the root delay approach by examining centrality, temporal and weather dimensions at the origin-destination airport. We also experiment with examining delay propagation using time-of-day factors and identification of airport community clusters. 

The state-of-the-art techniques used in modeling flight delay prediction problems include statistical analysis such as Logistic Regression (Govinda et al, 2017) where 80% accuracy in prediction was achieved. Probabilistic models such as Survival models in (Wong et al, 2012) were used to show that weather is the cause of delays that tends to result in longer departure delays. (Chakrabarty, 2019) compared the performance of machine learning approaches like random forest (Rebollo et al, 2014), support vector machine and gradient boosting classifiers in predicting arrival delay and concluded that GBC achieved the best performance of 85.7% accuracy. Temporal variables such as time-of-day and season, and spatial variables such as previous type of delay and OD-pair delay were identified as the most relevant variables for the delay classification problem  Most recently, graph-to-sequence deep learning frameworks like AG2S-Net have outperformed state-of-the-art models in multi-step ahead prediction in terms of MAE scores (Bao et al, 2021). 

We have chosen to apply Logistic Regression, Gradient Boosted Tree, Linear Support Vector Classifier, Random Forests, Ensemble Learning and a Multi-task Algorithm to the selected prediction problem. We are interested to assess the use of temporal variables and OD-pair delay variable with our algorithms and evaluate the results in our study. Our baseline for comparison is the 82% accuracy that could be achieved from simply predicting the majority class of non-delays. The main evaluation metric we have selected is the F2-score which places more emphasis on False Negatives.  $$F2score = \frac{((1 + 2^2) * Precision * Recall)} {(2^2 * Precision + Recall)}$$

Using F2-score aligns the evaluation of our algorithm effectiveness with our assessment that false negatives are more costly to airline passengers. We have also selected F1-score and Accuracy as two other evaluation metrics to facilitate comparison with other project studies.

#### III. EDA, Graph Analytics & Discussion of Challenges -- 

<a href="https://github.com/UCB-w261/w261-sp22-finalproject-team-t14/blob/main/Final_EDA_Team_14.ipynb">Link to EDA notebook </a>

<a href="https://github.com/UCB-w261/w261-sp22-finalproject-team-t14/blob/main/Weighted%20Page%20Rank%20from%20Scratch_team14_PhaseIV_Final_Submission.ipynb">Link to Graph Analytics notebook </a>

<a href="https://github.com/UCB-w261/w261-sp22-finalproject-team-t14/blob/main/Exploration%20and%20Graphs%20with%20Pyspark%20and%20GraphFrames.ipynb">Link to Other Graph Explorations notebook </a>

#### EDA

__Key Insights__

As discussed above, Bureau of Transport Statistics explains the five main causes delays.
We observe in the chart below that carrier delay and late aircraft delay are the two main drivers of flight delays. This could point us in the direction of trying to understand how delays propagate and accumulate over time in the same day due to late aircraft delay which spills over to the next flight or resulting airport capacity management issues which impact other flights.


![Delays](https://raw.githubusercontent.com/UCB-w261/w261-sp22-finalproject-team-t14/main/images/Cause_of_Delays.png?token=GHSAT0AAAAAABQFSMQSSF2T7VP3OLI4VL74YSYRUGQ)

Tier-two airlines like Frontier and Allegiant appear to have large proportions of flights which are delayed but the majority of airlines fall below the 15-minute delay threshold. There may not be a strong connection of airline variable with delay prediction except in the case of a few airline companies.

![Delay by Airline](https://raw.githubusercontent.com/UCB-w261/w261-sp22-finalproject-team-t14/main/images/2015_2019_Delays_by_Airline.png?token=GHSAT0AAAAAABR4TUCVSMCB5OHXXO37LTLAYSYSA3A)

Average delays tend to be highest in summer months such as June and above average in winter months. This is aligned with our research that summer thunderstorms and winter snow tends to reduce visibility and cloud ceiling height leading to delays. 

![Avg Delay](https://raw.githubusercontent.com/UCB-w261/w261-sp22-finalproject-team-t14/main/images/2015_2019_Avg_Flight_Delays.png?token=GHSAT0AAAAAABR4TUCUGHKDSXWJ5BFNFWVGYSYSBTQ)

As depicted below, the average delays in minutes peak in the morning from hours 2 to 3. The delays are lowest at 5-6 and steadily increase from hours 6 to 20.

![Avg Delay ToD](https://raw.githubusercontent.com/UCB-w261/w261-sp22-finalproject-team-t14/main/images/2015_2019_Time_of_Day_Delays.png?token=GHSAT0AAAAAABR4TUCVAXNWRRHMXOZYDFJOYSVU44Q)

The airport locations are depicted with red circles and their closest weather stations are blue circles. By mapping the airports and weather stations together we got a blended purple which indicates that the weather stations are close enough to the airports to get reliable weather data. Therefore, we can get the weather for the airport by using the closest weather station (Haversine distance approach).

![Weather Station Locations](https://raw.githubusercontent.com/UCB-w261/w261-sp22-finalproject-team-t14/main/images/Weather_Station_Closest_Airport.png?token=GHSAT0AAAAAABR4TUCUPRBNJG4YFGJMWOJUYSVU5QQ)


___Correlation Plot: Weather & Flights Datasets___

The correlation plot shows that arrival related data is closely correlated with delay related variables. We should consider removing arrival data to reduce multi-collinearity with any lagged delay variables and also to handle arrival variables with care to ensure data leakage does not occur.

![Correlation Plot](https://raw.githubusercontent.com/UCB-w261/w261-sp22-finalproject-team-t14/main/images/Correlation_Plot_Weather_Flights_Data.png?token=GHSAT0AAAAAABQFSMQTPOZE3MACA5MS24QAYSYTODQ)


__Missing Data__

__Flights__

In the 6-month flights dataset 53 out of 109 columns have more than 50% of their data missing (see bar plots below). These columns for the most part are delay type and diverted flight related fields. For the purposes of this problem, we recommend removing these columns.

![missing flights plot 1](https://raw.githubusercontent.com/UCB-w261/w261-sp22-finalproject-team-t14/main/images/Missing_Flight_Data_Percentage.png?token=GHSAT0AAAAAABQFSMQT2ZYVHTLWI6IMGU3KYSYRSXQ)
![missing flights plot 2](https://raw.githubusercontent.com/UCB-w261/w261-sp22-finalproject-team-t14/main/images/Missing_Flight_Data.png?token=GHSAT0AAAAAABQFSMQT5MUDO6MDBFLPCS72YSYRRQQ)

Figure 3.1, demonstrates there are duplicate records in the flight dataset caused by the `airlines_size_test.parquet` file. With this file included, the total records  are  63,493,682. We will need to remove these duplicates from the flights dataset prior to joining to the weather dataset to ensure we are only including the 31,746,841 flights.



___Figure 3.1 Duplicate Records in Flights Table___

| Name                             | Year            | Records          | Unique Records    |
| -------------------------------- | --------------- | ---------------- | ----------------- |
| 2015.parquet                     | 2015            |     5,819,079.00 |      5,819,079.00 |
| 2016.parquet                     | 2016            |     5,617,658.00 |      5,617,658.00 |
| 2017.parquet                     | 2017            |     5,674,621.00 |      5,674,621.00 |
| 2018.parquet                     | 2018            |     7,213,446.00 |      7,213,446.00 |
| 2019.parquet                     | 2019            |     7,422,037.00 |      7,422,037.00 |
| <br>airlines\_size\_test.parquet | Additional File |   31,746,841.00  |    31,746,841.00  |


__Weather__

We investigated the missing weather values from the Metar Aviation Routine Report (FM-15) that we are interested in including as features to our ML model. Figure 3.2 shows that Weather Stations with Automated Quality Control proceedures (V020) have more weather data values missing than Weather Stations that are subjected to a Quality Control check (V030). A majority of the values that are missing for the V020 weather stations are caused by missing values for Atmospheric Pressure which is a weather metric that is not mandatory for the FM-15 report.

___Figure 3.2: Missing Weather Values by Station___


| Quality Control                     | Average Missing Values by Station | Average Missing Values by Station       | Number of Stations |
| ----------------------------------- | --------------------------------- | --------------------------------------- | ------------------ |
| V020 - Automated Quality Control    | 5860.172613                       | 116.4778206                             | 2074               |
| V030 - Subjected to Quality Control | 42.18965517                       | 16.21336207                             | 464                |
| Notes:                              | _\* Excludes Wind Angle_            | _\* Excludes Wind Angle, Atmos. Pressure_ | __2538__        |

_Note: This is the average of the total missing values for the columns Wind Angle, Wind Speed, Ceiling Height, Visibility, Temperature, Dew Point, Atmospheric Pressure, Precipitation Hours, Precipitation by station_


As shown in Figure 3.3 below, most of the missing values passed the limit check but did not pass a quality control check. Additionally, the Wind Angle values that were missing did not meet quality assurance check, were suspect, or erroneous. Note: Missing wind angle values that did pass the quality assurance check occurred when Wind Speed is 0 and as a result there is no wind angle.

___Figure 3.3: Missing Weather Values by Quality Code___

| Quality                           | Wind Angle | Wind Speed | Ceiling Height | Visibility | Temperature | Dew Point | Atmospheric Pressure | Precipitation Hours | Precipitation |
| --------------------------------- | ---------- | ---------- | -------------- | ---------- | ----------- | --------- | -------------------- | ------------------- | ------------- |
| 1 - Passed QC Checks              | 96824      | 0          | 0              | 0          | 0           | 0         | 0                    | 0                   | 0             |
| 2 - Suspect                       | 0          | 0          | 0              | 0          | 0           | 0         | 0                    | 0                   | 0             |
| 5 - Passed QC NCEI data           | 3303454    | 0          | 0              | 0          | 0           | 0         | 0                    | 0                   | 0             |
| 6 - Suspect NCEI data             | 972        | 0          | 0              | 0          | 0           | 0         | 0                    | 0                   | 0             |
| 7 - Erroneous NCEI data           | 66         | 0          | 0              | 0          | 0           | 0         | 0                    | 0                   | 2             |
| 9 - Passed Limit Check            | 198265     | 198612     | 434460         | 0          | 173869      | 246804    | 11924476             | 0                   | 2292          |
| A - Flagged as Suspect            | 27         | 0          | 0              | 0          | 0           | 0         | 0                    | 0                   | 0             |
| I - Inserted by Validator         | 9          | 0          | 0              | 0          | 0           | 0         | 0                    | 0                   | 0             |
| P - Suspect replaced by Validator | 107        | 0          | 0              | 0          | 0           | 0         | 0                    | 0                   | 0             |
| U - Replaced Edited Value         | 10         | 0          | 0              | 0          | 0           | 0         | 0                    | 0                   | 0             |



<br>
<br>

#### Graph Analytics

A flight network can be represented by a Power-Law Distribution with uneven distributions of nodes and relationships. Most airports have few relationships but some airports have a lot which creates hub-and-spoke structures. Using a traditional analytics approach may obscure patterns due to the skew and hence graph analytics are used to keep the focus on relationships.

PageRank is the best-known centrality algorithm and helps us identify airport nodes which are most important in the flight traffic network. It considers the influence of a node’s neighbors, and their neighbors. For example, an airport which is connected to a few highly connected airport hubs would have more impact on transport delays than an airport connected to many small but less well-connected airports. PageRank is computed either by iteratively distributing one node’s rank over its neighbors or by randomly traversing the graph and counting the frequency with which each node is hit during these walks.

We implemented a weighted page rank from scratch and provide a geographic representation of influential points in the flight network.

The size of the circle represents the PageRank weighted using number of connections (flights).
We observe that the East region and some parts of the West have a high proportion of connections which represent major airport transport hubs.

![Top Airport Flight Connections](https://raw.githubusercontent.com/UCB-w261/w261-sp22-finalproject-team-t14/main/images/top_airports_connections.png?token=GHSAT0AAAAAABQFSMQTW3JLPCGFVY2QAL72YSYTSGA)

The size of the circle represents PageRank weighted using delays. 
We observe that the East Region in the US has a high proportion of airports with delays and that delays and volume of connections(flights) at each airport are correlated.

![Top Airports Delays](https://raw.githubusercontent.com/UCB-w261/w261-sp22-finalproject-team-t14/main/images/top_airports_delay.png?token=GHSAT0AAAAAABQFSMQSYL3VU6V76TJDKCNOYSYTTLA)

This is an example of Power Law Distribution where a small number of airports hubs hold disproportionately high influence on the transport network delays.

![Pareto Delays](https://raw.githubusercontent.com/UCB-w261/w261-sp22-finalproject-team-t14/main/images/pareto_delay.png?token=GHSAT0AAAAAABR4TUCVJBFOQL65ZCSOMCYCYSYUN5Q)

This is an example of Power Law Distribution where a small number of airports hubs hold disproportionately high influence on the transport network traffic.

![Pareto Connections](https://raw.githubusercontent.com/UCB-w261/w261-sp22-finalproject-team-t14/main/images/pareto_cnn.png?token=GHSAT0AAAAAABR4TUCUVXKM7OKDCOSQNWKWYSYUMXA)


In Community Detection Algorithms, we find airport communities and uncover hubs to study cascading effects of delays and weather.
We experimented with using Strongly Connected Components and Label Propagation to find clusters of airports with high connectivity and traffic that may propagate delays or be exposed to similar weather patterns. The Label Propagation algorithm (LPA) is a fast algorithm for finding communities in a graph. In LPA, nodes select their group based on their direct neighbors. This process is well suited to networks where groupings are less clear. Unfortunately, these algorithms failed due to long runtime.


#### Challenges

In the flights data around 50% of the variables had more than 50% of its data missing. This would require careful consideration of imputation of mean, rolling averages and unknown category to clean up the data based on our understanding of the data. We also noticed a number of categorical fields which would require conversion to numbers and one-hot encoding in future. In cancelled flights, we observed duplicates where flights were cancelled and then relaunched to a different location but that was not an issue since we decided to drop cancelled flights from our analysis. We found missing weather values represented by '9's and duplicate records based on reporting frequency. 

Additonally, the subject of predicting departures occurs in a time-series context meaning we would have to take extra care to ensure no data leakage occurs by excluding information occurring within 2 hours of or after departure such as departure delay by cause. We found that standardization of the time zones between the flights dataset and weather dataset was critical to be able to join data within a 1-hour window prior to the 2 hours before departure. Thus, a challenge we faced with joining the data was inconsistent keys and timestamps across the weather and flights datasets. The flights dataset had origin_airport_id and dest_airport_id keys which did not correspond to the weather dataset station_id key. Additionally, the flights dataset timestamp is in the airport’s local time zone, whereas the weather dataset is in the weathers stations GMT(UTC) time zone. Finally, as mentioned in the above EDA we have missing weather data that needs to be imputed which will present a challenge as we want to ensure that there is no data leakage.



#### Solutions

__Missing Data__

To handle missing data, we excluded all diverted flights and delay type columns, cancelled flights, and small airports from the final dataset. For the weather data, the LAST OBSERVATION CARRIED FORWARD (Forward Fill) and using the closest weather station to the airport to fill in missing data using only historical data within the window of 2-3 hours before the CRS flight departure time.  If weather values were missing upon joining on the closest weather station, we would use the next closest weather station data within a 6-mile radius. The LAST OBSERVATION CARRIED FORWARD approach is suitable for a real time model prediction and only requires past data. Finally, we dropped the duplicates in the flights dataset.

``` PYTHON
# Defining window function for forward fill (Last ObservationCarried forward)
fwd_fill = Window.partitionBy('airport_id').orderBy('airport_id','date','start_hr').rowsBetween(Window.unboundedPreceding,Window.currentRow)
```
__Performance__

For performance considerations, we converted any csv files we used to parquet files and stored them in the blob storage. We opted to use parquet because it is a columnar storage that allows for a faster reading of the data and when used in combination with Spark it can boost SQL performance by up to 10x. Additionally, we utilized checkpointing to save the data in blob storage after joins so that we do not have to re-run any computationally expensive joins.


Based on our analysis, we decided to move forward with the following features in our ML pipeline :

Categorical Features: QUARTER, MONTH, DAY_OF_MONTH, DAY_OF_WEEK, OP_CARRIER_AIRLINE_ID, ORIGIN_AIRPORT_ID, DEST_AIRPORT_ID, DEPARTURE_Hour_CRS

Numerical Features: DISTANCE, wind_speed_mps_orig, ceiling_ht_dim_orig, visibility_meters_orig, temp_cels_orig, dew_pt_orig, atmos_press_orig, precip_milimeters_orig, wind_speed_mps_dest, ceiling_ht_dim_dest, visibility_meters_dest, temp_cels_dest, dew_pt_dest, atmos_press_dest, precip_milimeters_dest

__Joins__

<a href="https://github.com/UCB-w261/w261-sp22-finalproject-team-t14/blob/main/Feature_Engineering_Joins.ipynb">Link to Joins notebook </a>

___Figure 3.3: Reference Table___

<img src="https://raw.githubusercontent.com/UCB-w261/w261-sp22-finalproject-team-t14/main/images/joins_part1.png?token=GHSAT0AAAAAABQFSMQTQMLPS37Z4W2IHW7QYSYTYFA" width="800"/>

In order to combine the `weather` and `flights` tables together we created a reference table to aid us in the joining process. Specifically, we created a table called airport_weather_ref which maps the airports to a specific timezone and the closest weather station (using Haversine Distance Formula see formula below) by joining the flights, timezone, and station temporary tables. Using this approach, the furthest weather station from an airport was 10 kilometers and the average distance from a weather station was .867 kilometers. The timezone from the airport_weather_ref is used to convert the timestamp in the airlines data to UTC so that is consistent with the weather table for joining purposes. Finally, using this airport_weather_ref table we were able to join the airline and weather tables together based on a two hour time horizon window (i.e. weather data is joined on a window 2-3 hours before the expected flight departure). See formula and code snippet below for additional details on the reference table and Haversine Distance Formula. 

The Haversine Distance formula is as follows:

$$ distance\ kilometers = 2rsin^{-1}\sqrt{({sin}^2\frac{\Phi_2-\Phi_1}{2})+cos{\left(\Phi_1\right)cos(\Phi_2)sin^2}(\frac{\lambda_2-\lambda_1}{2})}$$

where r is the radius of the earth (6371 kilometers)




``` sql

select distinct t1.airport_id, 
                t2.station_id, 
                t1.Lat as air_Lat, 
                t1.Long as air_Long, 
                t2.lon, 
                t2.lat, 
                acos(
                sin(radians(t1.Lat)) * sin(radians(t2.lat)) + 
                cos(radians(t1.Lat)) * cos(radians(t2.lat)) * 
                cos(radians(t1.Long) - radians(t2.lon))
                ) * (6371.0) as dist_km,
                t3.tz_database as time_zone
from air_locs as t1 
left join w_stats as t2 on t1.state = t2.state
left join tz as t3 on t1.airport = t3.IATA

```

#### IV. Feature Engineering
<a href="https://github.com/UCB-w261/w261-sp22-finalproject-team-t14/blob/main/Feature_Engineering_Joins.ipynb">Link to Feature Engineering notebook </a>


In our model pipeline we applied relevant feature transformations such as One Hot Encoding (OHE) categorical variables (quarter, month, etc.) and normalization of numeric variables (distance, wind speed, etc). For Machine Learning algorithms such as Linear Regression, Logistic Regression, Support Vector Machine,  OHE is required to convert categorical variables into a numeric form. Additionally, standardizing numeric features by subtracting the mean and dividing the standard deviation is required for most algorithms to ensure that numeric data is on the same scale. Moreover, standardizing numeric data helps the models converge more quickly and allows the objective function to work.

Please see _Algorithm Exploration_ section part _a_ for additional details on feature transformations.

In total, we created 10 engineered features which include both time (OD Delay Pair, Rolling 90 Day Avg, etc.) and graph based features (Air Page Rank Traffic, Coelesced PgRank, etc.).

<b> Continuous Variables </b> 

___OD Delay Pair___ : This is the prior 2-3 hours median delay for each origin-destination airport pair date-time hour block in minutes. We used origin_airport_id, departure_delay_new and dest_airport_id to create an OD-median delay pair column then created a new column where each row will use OD-delay information from the one hour block starting 3 hours earlier. Any null values were filled using forward fill for the day and if not found then forward fill for all prior days in history. We chose to create this variable based on our literature review where research showed this feature was significant in explaining delays.

___Rolling Ninety Day Average___: This is the average prior 90 day average delay in minutes for each origin airport. An accumulating sliding window function was used. For example, records on Jan 2, 2015 would have an average using 1 day of prior data from Jan 1, 2015. The sliding window of prior day averages would grow in length till 90 days and then the window would start to slide as we moved from current to next day. A small 0.05% of the population had null values as these were the first flights from the origin in the dataset. We chose to create this temporal feature based on the understanding that there might be structural and operational deficiencies in certain airport locations that would be reflected in historical flight delay metrics.

___Air Page Rank Traffic___: This is simple unweighted Page Rank feature based on traffic for each origin airport based on origin-destination flights. We chose to create this metric because graph-based features better reflect the underlying relationships between airports particularly when there is a skew in distribution where certain airports and regions have the most influence on the flight network and this could have great predictive power for our model.

___Coalesced PgRank Origin___: This is a feature generated from a weighted Page Rank implemented from scratch. The Page Rank is weighted by departure delays in minutes from the airport and page rank is executed for 35 splits of the data by year (5 years) and time-of-day (7 parts of the day). A computed page rank is then lagged by a year for 2016-2019 within each time of day category while average weighted page rank by delay for 2015-2019 is imputed for 2015 records. This is then joined to the main dataset by origin airport, year and time of day. We were interested to create this page rank metric weighted by prior delay to emphasize airports which have disproportionate influence in delays across the network and to use the time of day splits to prove our theory that airport delays accumulate over the course of the day and reset at the start of the next day.

___Coalesced PgRank Destination___: Similar to the feature above where page rank weighted by delay, split by time of day and year and then lagged is implemented from scratch. The only difference in this case is that this is joined to the main dataset by destination airport, year and time of day. 

___Connection Ranking Origin___: This is a feature generated from a weighted Page Rank implemented from scratch. The Page Rank is weighted by connections, defined as number of flights from the airport and page rank is executed for 35 splits of the data by year (5 years) and time-of-day (7 parts of the day). A computed page rank is then lagged by a year for 2016-2019 within each time of day category while average weighted page rank by connections for 2015-2019 is imputed for 2015 records. This is then joined to the main dataset by origin airport, year and time of day. We created this feature to understand if page rank weighted by prior traffic and split by time of day would produce different results compared to page rank weighted by prior delays. If yes, this might be an opportunity to study these differences and include in our predictive model. We found that the results were somewhat similar. 

___Connection Ranking Destination___: Similar to the feature above where page rank weighted by connections (number of flights), split by time of day and year and then lagged is implemented from scratch. The only difference in this case is that this is joined to the main dataset by destination airport, year and time of day.


##### Categorical Variables  

___Time of day___: This was divided in seven different time slots. Category 1 represents `Morning`: 6-8 hrs, Category 2 represents `Mid-Morning` : 9-10 hrs, Category 3 represents `Mid-Day`: 11-13 hrs, Category 4 represents `Early Afternoon` : 14-16 hrs, Category 5 represents `Late Afternoon` : 17-19 hrs, Category 6 represents `Evening`: 20-22 hrs, Category 7 represents `Night`: 23-24 hrs and 0-5hrs. This variable was used by other studies in our literature review. Additionally, our EDA shows that delays accumulate through the passing of time during the day, we decided to create this variable to see if this could give our model additional predictive power or create different models for different time of day.

___Season___: The year was divided in four seasons. Category 1 represents `Spring`: months 3-5; Category 2 represents`Summer`: months 6-8; Category 4 represents `Fall`: months 9-11; Category 5 represents `Winter`: months 12,1,2. Our research showed that weather patterns that affect take-off and landing times were seasonal and we created this variable to give our model additional predictive power.

___WKDAY___: 1 represents weekdays from Monday to Friday and 0 represents weekends from Saturday to Sunday. We created this variable based on the understanding that weekend flights tend to be more in demand for leisure and the added passenger volume could influence delays and provide additional predictive power for our model.

#### V. Algorithm Exploration

<a href="https://github.com/UCB-w261/w261-sp22-finalproject-team-t14/blob/main/team14_pipeline_implementation.ipynb"> Link to Pipeline Implementation Notebook </a>

We explored four different algorithms - Logistic Regression (LogR), Gradient Boosted Tree (GBT), Linear Support Vector Classifier (LSVC), and Random Forest(RF).  This section is divided in  the following sections:   

> a. Data Preprocessing  
b. Pipeline Structure  
c. Algorithm Evaluation Metrics  
d. Algorithms Testing and Best Algorithm Selection  
e. Key Attributes Identification  
f. Exploring Best Algorithm  
g. Further Optimizing Selected Algorithm  
h. Ensemble Algorithm  
i. Conclusion

##### a. Data Pre-Processing:

Seven steps were implemented to prepare the data for the models. The steps were:
1. Define Input and Output variables
2. Manage Imbalanced Data
3. Weight Column  
4. Split Data for Train, Validation and Testing  
5. Folds for Custom Time Series Grid Search  
6. Define Indexers, OHES, Imputers and Scalers
7. Implement Custom Grid Search for Time Series Data

These steps are discussed in the following paragraphs.

**(1) Define input and output variables.**  The following variables were included:

- **Categorical Input variables:**  11 categorical variables.

`['QUARTER', 'MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK', 'OP_CARRIER_AIRLINE_ID', 'ORIGIN_AIRPORT_ID', 'DEST_AIRPORT_ID', 'SEASON', 'WKDAY', 'DEPARTURE_Hour_CRS', 'time_of_day_int']
`
- **Numeric Input Variables:** 22 numeric variables.

`['DISTANCE', 'wind_speed_mps_orig', 'ceiling_ht_dim_orig', 'visibility_meters_orig', 'temp_cels_orig', 'dew_pt_orig', 'atmos_press_orig', 'precip_milimeters_orig', `
`'wind_speed_mps_dest', 'ceiling_ht_dim_dest', 'visibility_meters_dest', 'temp_cels_dest', 'dew_pt_dest', 'atmos_press_dest', 'precip_milimeters_dest', `
`'rolling_ninety_day_average', 'Air_Page_Rank_traffic','OD_delay_pair', 'Coalesced_PgRank_orig', 'Coalesced_PgRank_dest', 'Conn_Ranking_orig', 'Conn_Ranking_dest']
`  

- **Output Variable:** ‘DEP_DEL15’ – binary variable where `one` indicates the flight was delayed (> 15 minutes), and `zero` indicates it was not.

**(2) Managing Imbalanced Data:**  

The flight data is imbalanced as 82% of the flights are on time, and only 18% of the flights are delayed. As imbalanced data has a negative impact on model accuracy, three different strategies to manage it were explored:  

- `Oversampling Minority Class`, 
- `Under sampling Majority Class`, and 
- `Balanced Approach` (combining oversampling and under sampling while maintaining same number of observations.  

We selected `Under Sampling` after testing the alternatives with 2.5% of the data.  We observed slightly better model performance when using `Under Sampling` versus the other approaches.  In addition, time to train the model was reduced significantly. One important disadvantage for `Under Sampling` is the significant reduction in the amount of data, however we believe the benefits outweigh this disadvantage.

**(3) Weight Column:**  
Most models in `PySpark` offer an alternative to manage imbalance data by including a `weight` column which reflects the relative weight of the minority class versus the majority class.  The weight of observations in a Class_i is defined by the following formula (where C is number of classes):

$$
\text{Weight Class}_i = \frac{ \text{Total Count}}{C * \text{Class Count}_i }
$$

The formula for the weights and the approach was taken from the following post - https://www.datatrigger.org/post/spark_3_weighted_random_forest/

While we maintained a column with `Weights`, our preliminary assesment indicates that `Under Sampling` provides better results when compared to using `Weight`.  Thus, we will focus on `Under Sampling` for managing Imbalance Data.

**(4) Split Data for Train, Validation and Testing:**  the data was divided in the following way:
- Train and Validation Data = 2015 to 2018.
- Test Data = 2019

**(5) Folds for Custom Time Series Grid Search:** 
Two alternatives for splitting data for time series `Grid Search` were explored – `sliding window` and `moving window`.  As we wanted to retain the seasonality of a year, we divided the folds by year.  However, given the limited number of years (2015 to 2018), we felt sliding window provided enough splits for a proper implementation of Grid Search.  We understand the risk of potential leakage, but we decided in favor of having enough folds (data) to implement a grid search. The folds used for `Grid Search` are as follows: 

- Fold 1: Train = 2015, Validation = 2016
- Fold 2: Train = 2015-2016, Validation = 2017
- Fold 3: Train = 2015-2017, Validation = 2018.

**(6) Define Indexers, One-Hot Encoders(OHES), Imputers and Scalers:**
For `Categorical Data` we defined `Indexers` and `One Hot Encoding`, and for `Numeric Data` we defined `Imputers` (using mean) and `StandardScalers` using the `mean` and `standard deviation` to normalize numeric features. 

The following code presents how Indexers, OHES, Imputers and Scalers were implemented:
```shell
indexers = map(lambda c: StringIndexer(inputCol=c, outputCol=c+"_idx", handleInvalid = 'keep'), categoricals)
ohes = map(lambda c: OneHotEncoder(inputCol=c + "_idx", outputCol=c+"_class", dropLast=True),categoricals)
imputers = Imputer(inputCols = numerics, outputCols = numerics)

# Establish features columns
featureCols = list(map(lambda c: c+"_class", categoricals)) + numerics

model_matrix_stages = list(indexers) + list(ohes) + [imputers] + \
                     [VectorAssembler(inputCols=featureCols, outputCol="features")]

# Apply StandardScaler to create scaledFeatures
scaler = StandardScaler(inputCol="features",
                        outputCol="scaledFeatures",
                        withStd=True,
                        withMean=True)
```
**(7) Implement Custom Grid Search for Time Series Data:** We first implemented a triple loop to test different combinations of parameters for a given model, optimizing by F2_score.  Our implementation helped us identify combinations of parameters, but it was dependent on the type of model.  However, as Lea Cleary shared her `Custom Grid Search` function, we integrated it into our pipeline.  Lea’s custom function gave us flexibility to implement Grid Search with all the algorithms that we wanted to explore.

##### b. Pipeline Structure

The key elements of the ‘Grid Search’ pipeline are as follows (assuming `model_matrix_stage`s and `scaler` were already defined as presented in previous section):

- Define the model – example for Random Forest  

```

    rf = RandomForestClassifier(featuresCol = 'scaledFeatures', labelCol = 'label',
                                featureSubsetStrategy='auto', 
                                impurity='gini',
                                seed=123)
```

- Set the Grid Search set of parameters

```
    grid = ParamGridBuilder()\
                .addGrid(md.maxDepth, [5, 8, 10, 14])\
                .addGrid(md. numTrees, [50, 110, 150, 200, 250])\
                .build()
```

- Define Pipeline for the model
```
pipeline = Pipeline(stages=model_matrix_stages+[scaler]+[rf])
```

- Define evaluator

```
  evaluator = BinaryClassificationEvaluator()
```

- Define CrossValidator for model tuning

```
  crossval = CustomCrossValidator(estimator=pipeline,
                                  estimatorParamMaps=grid,
                                  evaluator=evaluator,
                                  splitWord = ('train', 'test'),
                                  cvCol = 'cv',
                                  parallelism=4)
```

- Fit the Cross Validation to the Data Segments (folds), and select the best model

```
  Pipeline_rf = crossval.fit(data_segments)

  rf_model = Pipeline_rf.bestModel
```

- Generate Predictions from model using test data

```
pred = rf_model.transform(test).select("DEP_DEL15", "prediction")
```

- Get metrics based on predictions

```
metricsT = MulticlassMetrics(pred.rdd.map(lambda x: (x[1], x[0])))
```

##### c. Algorithm Evaluation Metrics: 
To evaluate the models, we calculated key metrics such as `Accuracy`, `Precision`, `Recall`, `F1_score`, `F05_score`, and `F2_score`.

However, given the nature of the data and the business question, we decided to focus on `F2_score` (which gives more weight to Recall), `F1_score` and `Accuracy`.

A **Positive** event is defined as a flight "being delayed".  Based on this definition, a `False Positive` is a prediction for a flight being delayed when it was not, while a `False Negative` is a prediction that a flight was NOT delayed, when it was.  The potential costs for these events are:

**False Positive Cost**:
- Passenger get's stressed and may start looking for alternatives.   
- Passenger may start complaining (even before the delay is confirmed or not)
- Passenger may try to cancel (it he/she has flexibility)
- May lead to some frustration. However, if flight is not delayed, he/she may be pleasantly surprised.

**False Negative Cost:** 
- Passenger is frustrated. He/she thought things were OK, and now the flight is delayed.  No time to change anything.
- Passenger feels helpless - he/she can't do anything to manage situation.
- Reputation cost for airline and/or company providing service.

Based on these costs, we identified `False Negative` as a more costly event with strong reputational cost (among other costs).  As a result of this, as we evaluate the models, we will use `Evaluation Metrics` that place emphasys on `False Negatives`. Thus, we will integrate `F2_score` as a key metric to evaluate models together with `Accuracy` and `F1_score`.

F2_score is a variation of Fbeta_score.  The formula for Fbeta_score is as follows:

$$
Fbeta = \frac{((1 + beta^2) * Precision * Recall)} {(beta^2 * Precision )+ Recall)}
$$

Where: 
>- F0.5-Measure (beta=0.5): Places more weight on precision, less weight on recall.
>- F1-Measure (beta=1.0): Balance the weight on precision and recall.
>- F2-Measure (beta=2.0): Places less weight on precision, more weight on recall

Definitions for `FBeta_score` were taken from the following post -https://machinelearningmastery.com/fbeta-measure-for-machine-learning/#:~:text=The%20F2%2Dmeasure%20is%20calculated,(4%20*%20Precision%20%2B%20Recall)

##### d. Algorithms Testing and Best Algorithm Selection 

The process for selecting the best model is as follows: 
>1. Find optimal parameter leveraging a cross validation, using 5% of the data (5% of data was used to speed up the process). 
2. Use optimal parameters found in Grid Search Process to generate a final model using the full data.
3. Select the best model based on performance (Accuracy, F2_score, F1_score).

The following algorithms were explored: `Logistic Regression` (LogR), `Gradient Boosted Tree` (GBT), `Linear Support Vector Classifier` (LSVC), and `Random Forest` (RF).  Some factors that guided this decision were:

- These algorithms had a strong implementation in PySpark.
- Logistic Regression is a basic algorithm which could help to establish a base value for performance.
- We will implement Logistic Regression from Scratch, thus we are interested in evaluating Logistic Regression performance.
- These algorithms had efficient implementation in PySpark with parallelized implementation which provided reasonable times for training and prediction. 
- Random Forest offered several parameters to optimize (depth, number of trees, weight) which we thought would give us space for further optimization.
- We plan to use Random Forest to identify key features driving model performance.

Using 5% of the final joined data, we implemented grid search for each algorithm.  The parameters used for the optimization were as follows:

___Figure 5.1: Parameters for Grid Search.___

| Algorithm | Parameter 1                    | Parameter 2                     |
| --------- | ------------------------------ | ------------------------------- |
| LogR      | regParam, \[0.1, 0.05, 0.01\]) | elasticNetParam, \[0, 0.5, 1\]) |
| GBT       | maxDepth, \[5, 8, 10\]         | maxIter, \[10, 15, 20\]         |
| LSVC      | regParam, \[0.1, 0.05, 0.01\]  | No additional Parameter         |
| RF        | maxDepth, \[10, 12, 14\]       | numTrees, \[110, 200, 250\]     |`

After running the grid search process, the configuration for the best performing models for each algorithm were as follows:
  
___Figure 5.2: Grid Selected Parameters and Grid Search Accuracy.___

| Algorithm | Grid Selected Parameters               | Grid Search Perform. |
| --------- | -------------------------------------- | -------------------- |
| LogR      | regParam = 0.01, elasticNetParam = 0.5 | 68.6%                |
| GBT       | maxDepth=5, maxIter=20                 | 70.9%                |
| LSVC      | regParam=0.01                          | 68.3%                |
| RF        | maxDepth=14, numTrees = 250,           | 71.3%                |`

Based on the Grid Search Performance metric, the best model is `Random Forest (RF)` with a grid search metric of 71.3%.

While the best Random Forest configuration found in Grid Search was maxDepth=14, numTrees = 250, when implementing this configuration with the full data (using under sampling, including 33 attributes), the process crashed several times.  We `unpersisted` all dataframes, but it did not help.  Thus, we decided to try the second best configuration which was maxDepth=10, numTrees = 200.   

To speed up the process, we proceeded to identify the important attributes leveraging the Random Forest attribute importance output. We identified the top 19 attributes representing 95% of the weight.  When fitting the models with these reduced attributes, the model performance was very similar to models with 33 attributes.  Thus, we decided to continue our analysis with these 19 attributes. We implemented a Grid Search for Random Forest using 19 variables with the following parameters:
- maxDepth, [5, 8, 10, 12, 14]
- numTrees, [80, 100, 110, 150]

The final best Random Forest model was - Number of Trees = 110, and maxDepth = 14.  We would use this model configuration for further exploration and optimization.

The table below presents the performance of the models using the optimized attributes (found in Grid Search), including the top 19 attributes) tested on the full data:

___Figure 5.3: Performance Metrics for basic algorithms___ 

|            | LR\_test | GBT\_test | LSVC test | RF\_test |
| ---------- | -------- | --------- | --------- | -------- |
| Accuracy   | 59.6%    | 68.3%     | 57.7%     | 64.8%    |
| Precision  | 27.1%    | 31.7%     | 26.1%     | 29.7%    |
| Recall     | 68.7%    | 60.3%     | 69.1%     | 64.8%    |
| F1\_Score  | 38.9%    | 41.5%     | 37.9%     | 40.8%    |
| F05\_Score | 30.8%    | 35.0%     | 29.8%     | 33.4%    |
| F2\_Score  | 52.6%    | 51.1%     | 52.0%     | 52.5%    |`

While Random Forest did not have the best performance when using the full data, we decided to continue with `Random Forest` to move forward as it offered a reasonable balance between F2_score (52.5%), Recall (64.8%) and Accuracy (64.8%). We also felt that Random Forest offered the opportunity to further fine tune parameters for increased performance.

We have seen a significant decrease in Precision Score when evaluating the model on imbalanced test data while training and validating the model on balanced data. While the Recall score tends to be high when testing on imbalanced data, the F1 score tends to be low as the low precision score drives it down.

##### e. Key Attributes Identification

To identify key attributes (out of the 33 attributes included), we first explored PCA.  However, we decided to focus on leveraging results from Random Forest for identifying and selecting key features (attributes).  Some key findings from our Principal Component Analysis are:

- As each level for a categorical variable is represented by a column, the number of variables for the principal components increases to around 754. We were able to map the variables to each Principal Component, but this made it challenging to interpret the relationship of the attributes and the principal components.
- 150 principal components represented 95% of the variance which represented a significant reduction of variables.
- When using 150 principal components, the accuracy of the model decreased by a percentage point. As it would be discussed later, we were able to reduce variables identified in our Random Forest implementation while retaining most of the accuracy of the full model.  

In summary, we felt PCA did not provide clear insights about key features, and selecting 150 Principal Components(PCs), which represent ~95% of variance, negatively impacted model performance.

PCA analysis is included in the `pipeline notebook`.

To identify key attributes, we leveraged the output from Random Forest. The following table presents the importance of key group of attributes (features).

___Figure 5.11: Key Attributes and Relative Importance as % of total.___

| Num Feat. | Type of Feature         | Importance |
| --------- | ----------------------- | ---------- |
| 10        | Engineered Feature      | 53.7%      |
| 5         | Time Related Feature    | 21.7%      |
| 14        | Weather Related Feature | 18.4%      |
| 1         | Airline                 | 3.5%       |
| 2         | Airport Related         | 2.4%       |
| 1         | Distance                | 0.3%       |
| 33        | TOTAL                   | 100.0%     |`

We created 10 features (Engineered Features) that had an importance of 53.7%, while `Weather Related Features` only had an importance of 18.4%, and Time related Features (quarter, month, day, time of day, day of week) had a relative importance of 21.7%. The following table presents the key attributes driving the importance for the Engineered Features.

___Figure 5.12: Key Engineered Features___

| Engineered Feature | Importance |
| ------------------ | ---------- |
| OD\_delay\_pair    | 23.1%      |
| time\_of\_day\_int | 22.1%      |
| Season             | 3.5%       |
| Page Rank Related  | 3.3%       |
| Others             | 1.7%       |
| Total              | 53.7%      |`

While `OD Delay Pair` and `Time of the Day` were important features for the model, the contribution of `Page Related Features` to model performance was relatively small.  Though we created page rank features per year and time of the day, it was not frequent enough to have a significant explanatory power. We successfully implemented simple unweighted PageRank in GraphFrames. However, we failed to implement weighted PageRank using NetworkX. Hence, we decided to create a weighted PageRank from scratch. Unfortunately, investing a significant amount of time implementing weighted Page Rank from scratch did not result in highly important engineered features.  We decided to implement Page Rank from scratch as it gave us flexibility to adjust weight by number of connections and amount of delay. 

Additionally, we also experimented with creating engineered features based on other Centrality Algorithms like Closeness Centrality with Pyspark udfs and Community Detection Algorithms like Label Propagation Algorithm (LPA) and Strongly Connected Components (SCC) but were not successful due to lack of memory and a timeout due to long runtime. See <a href="https://github.com/UCB-w261/w261-sp22-finalproject-team-t14/blob/main/Exploration%20and%20Graphs%20with%20Pyspark%20and%20GraphFrames.ipynb"> Exploration and Graphs with Pyspark and GraphFrames Notebook </a>.

The key feature for `Time Related Features` is `DEPARTURE_Hour` (1 to 24 hours) with a weight of 19.6%.

The following table present the key features related to `Weather`.  The top 4 features are related to weather in the Origin Airport.

___Figure 5.13: Top Weather Related Features.___

| Weather Related Feature  | Importance |
| ------------------------ | ---------- |
| wind\_speed\_mps\_orig   | 2.3%       |
| temp\_cels\_orig         | 2.2%       |
| precip\_milimeters\_orig | 2.2%       |
| ceiling\_ht\_dim\_orig   | 2.0%       |
| wind\_speed\_mps\_dest   | 1.9%       |
| Others                   | 7.8%       |
| Total                    | 18.4%      |`

Surprisingly, this shows that weather-related features were not individually as impactful as we expected.

##### e. Explore Best Algorithm

For the selected algorithm (Random Forest, maxDepth=14, numTrees=110), we explored the performance across the following metrics: by Season (1 to 4), by Day of the Week (1 to 8), by Day of the Month (1 to 31), by time of the day (1 to 7), by Workday (weekend vs. weekday), by Type of Airport based on Ranking by Amount of Delays (Group 1 to 4), by Type of Airport based on Ranking by Number of Connections (Group 1 to 4), by month of the year (1 to 12).  

The objective of this analysis was to identify specific segments (or attributes) where the algorithm performed best (or worst) which would indicate the potential for having independent models for the specific segment (or attribute).  The attributes that provided the best insights were `Time of the Day`, `Season`, and `Airport Ranking by Number of Connections`.  We will present a brief description of this results in the following paragraphs.

The `Time of the Day` was divided in seven different time slots which were defined as follows - `Morning`: 6-8 hrs, `Mid-Morning`: 9-10 hrs, `Mid-Day`: 11-13 hrs, `Early Afternoon`: 14-16 hrs, `Late Afternoon`: 17-19 hrs, `Evening`: 20-22 hrs, `Night`: 23-24 hrs and 0-5hrs . The following table presents the distribution of all flights per Time of day, and the % of those flights that are delayed.

___Figure 5.4: Distribution of flights by Time of the day.___

| Time of Day     | % of Total | % Delayed |
| --------------- | ---------- | --------- |
| Morning         | 20.5%      | 8.6%      |
| Mid Morning     | 12.2%      | 13.8%     |
| Mid Day         | 18.2%      | 17.5%     |
| Early Afternoon | 17.6%      | 22.1%     |
| Late Afternoon  | 17.7%      | 26.1%     |
| Evening         | 10.4%      | 26.0%     |
| Night           | 3.4%       | 10.6%     |
| Total           | 100.0%     | 18.2%     |`

While `Morning` flights represent 20.5% of total, they are usually on time (on higher proportion) as only 8.6% are delayed.  The time of the day with the highest proportion of delayed flights are `Later Afternoon` and `Evening`. The analysis suggests that the delays accumulate as the day progresses and resets at the beginning of the day. This motivated the inclusion of time-of-day variable in our modeling.

The following table presents the model's performance metrics for each time of the day using balanced train-val data.  

___Figure 5.5: Model performance by time of the day.___

| Metrics   | Morning | Mid Morning | Mid Day | Early Afternoon | Late Afternoon | Evening | Night |
| --------- | ------- | ----------- | ------- | --------------- | -------------- | ------- | ----- |
| Accuracy  | 69.2%   | 62.0%       | 60.3%   | 62.7%           | 65.3%          | 67.5%   | 69.1% |
| Precision | 71.3%   | 59.3%       | 62.1%   | 65.8%           | 65.6%          | 68.3%   | 64.1% |
| Recall    | 1.9%    | 35.5%       | 51.1%   | 71.8%           | 92.5%          | 88.4%   | 29.4% |
| F1 Score  | 3.8%    | 44.4%       | 56.0%   | 68.7%           | 76.7%          | 77.0%   | 40.3% |
| F05 Score | 8.7%    | 52.3%       | 59.5%   | 67.0%           | 69.6%          | 71.5%   | 51.9% |
| F2 Score  | 2.4%    | 38.6%       | 52.9%   | 70.5%           | 85.5%          | 83.5%   | 33.0% |`

  
There are important differences in performance across the different times of the day where the worst `Accuracy` is in `Mid_Day` and `Early Afternoon`, and the best `Accuracy` is at Night and in the Morning. In addition, the `Morning` and `Night` have some of the lowest F2_scores.  There may be an opportunity to further develop models to improve predictions by the different times of the day.

In relation to `Seasons`, the year was divided in four seasons defined as - `Spring`: months 3-5; `Summer`: months 6-8; `Fall`: months 9-11; `Winter`: months 12,1,2. The following table present the distribution of flights across seasons, and the pecentage of delayed flights per season.  All seasons have similar proportion of flights, however, flights in `Fall` have a relatively lower proportion of delayed flights (14.3%), while `Summer` has the largest proportion of delayed flights `21.6%`.

___Figure 5.6: Flight distribution per season and proportion of delayed flights.___

| Season | % of Total*| % Delayed |
| ------ | ---------- | --------- |
| Spring | 25.3%      | 17.8%     |
| Summer | 26.5%      | 21.6%     |
| Fall   | 25.1%      | 14.3%     |
| Winter | 23.2%      | 19.0%     |
| Total  | 100.0%     | 18.2%     |` 

*% Distribution of Total Flights

The following table presents the performance metrics by season using balanced train-validation data.

___Figure 5.7: Model performance by season.___

| Metric     | Spring | Summer | Fall  | Winter |
| ---------- | ------ | ------ | ----- | ------ |
| Accuracy   | 64.0%  | 65.2%  | 64.9% | 64.2%  |
| Precision  | 63.9%  | 65.9%  | 63.9% | 66.9%  |
| Recall     | 66.0%  | 77.5%  | 44.5% | 63.2%  |
| F1\_Score  | 64.9%  | 71.2%  | 52.4% | 65.0%  |
| F05\_Score | 64.3%  | 68.0%  | 58.7% | 66.1%  |
| F2\_Score  | 65.6%  | 74.8%  | 47.3% | 63.9%  |`

The best `Accuracy` and `F2_score` was for Summer, while the lowest `F2_score` is in fall.  Though the metrics are relatively close, there is also an opportunity to fine tune independent models by season of the year.

In relation to model performance by `Group of Airports based on Connections` (connections defined as number of flights), there are also opportunities for developing independent models by group of airlines.  These groups were created using the overall page rank by number of connections in the period 2015 to 2019.  This ranking was created as an indicator to classify Airports based on their importance to the Graph in term of number of connections. Group 1 includes the top 5% of airports based on Pag Rank by Connections, Group 2 includes the next 20% of airports, Group 3 include the next 25% of airports, and Group 4 include the remaining 50% of airports.   

The following table presents the distribution of airports based on PageRank weighted by number of connections.  Group 1 includes only 17 airports which represent 50.5% of all connections (flights) in the graph. Group 1 also has a slighly higher proportion of delayed flights (18.9%), and there may be an opportunity to improve performance in this group of airports.

___Figure 5.8: Airport Distribution based on PageRank by number of connections.___

| Group | \# Airports      | \# Connect(mil.)| % Total Conn | % Delayed |
| ----- | ---------------- | -------------- | ------------ | --------- |
| 1     | 17               | 14.8           | 50.5%        | 18.9%     |
| 2     | 65               | 11.6           | 39.9%        | 18.0%     |
| 3     | 82               | 2.0            | 7.0%         | 15.5%     |
| 4     | 164              | 0.8            | 2.7%         | 14.3%     |
| Total | 328              | 29.2           | 100.0%       | 18.2%     |`

The following table presents the performance of the selected Random Forest model by Group of Airports using balanced train-valid data.

___Figure 5.9: Model Performance by Group of Airports (based on number of connections)___

| Metrics   | Group 1 | Group 2 | Group 3 | Group 4 |
| --------- | ------- | ------- | ------- | ------- |
| Accuracy  | 63.9%   | 65.7%   | 64.3%   | 62.7%   |
| Precision | 65.3%   | 65.5%   | 65.5%   | 61.4%   |
| Recall    | 66.1%   | 66.0%   | 51.3%   | 48.7%   |
| F1 Score  | 65.7%   | 65.8%   | 57.5%   | 54.3%   |
| F05 Score | 65.5%   | 65.6%   | 62.0%   | 58.3%   |
| F2 Score  | 66.0%   | 65.9%   | 53.6%   | 50.8%   |`


The table indicates slightly better model performance (Accuracy and F2_score) for airports in Group 2.  However, there is significant opportunity to improve model performance in Group 1 which represents 50% of all connections in the graph.

As a reference, the following table presents summary information for PageRanking and Number of Connections for the top 17 airports (Group 1).

___Figure 5.10: Top 17 airports by PageRank based on Connections (Group 1)___.

| Rank | airport | Connec\_Ranking | Num\_connection | ORIGIN | ORIGIN\_CITY\_NAME    |
| ---- | ------- | --------------- | --------------- | ------ | --------------------- |
| 1    | 10397   | 0.05949         | 1,865,310       | ATL    | Atlanta, GA           |
| 2    | 13930   | 0.04876         | 1,428,494       | ORD    | Chicago, IL           |
| 3    | 11298   | 0.04188         | 1,158,829       | DFW    | Dallas/Fort Worth, TX |
| 4    | 11292   | 0.03573         | 1,090,882       | DEN    | Denver, CO            |
| 5    | 12892   | 0.02811         | 1,045,707       | LAX    | Los Angeles, CA       |
| 6    | 14771   | 0.02436         | 819,507         | SFO    | San Francisco, CA     |
| 7    | 11057   | 0.02386         | 759,787         | CLT    | Charlotte, NC         |
| 8    | 13487   | 0.02372         | 659,920         | MSP    | Minneapolis, MN       |
| 9    | 11433   | 0.02351         | 657,040         | DTW    | Detroit, MI           |
| 10   | 12266   | 0.02267         | 744,566         | IAH    | Houston, TX           |
| 11   | 14107   | 0.02201         | 787,700         | PHX    | Phoenix, AZ           |
| 12   | 12889   | 0.01964         | 731,648         | LAS    | Las Vegas, NV         |
| 13   | 14747   | 0.01928         | 642,030         | SEA    | Seattle, WA           |
| 14   | 14869   | 0.01888         | 501,813         | SLC    | Salt Lake City, UT    |
| 15   | 13204   | 0.01697         | 623,649         | MCO    | Orlando, FL           |
| 16   | 12953   | 0.01677         | 605,224         | LGA    | New York, NY          |
| 17   | 10721   | 0.01645         | 630,955         | BOS    | Boston, MA            |`

##### g. Further Optimizing Selected Algorithm
We explored Random Forest models including the top 19 features (seven categorical features and 12 numeric features) representing a total weight of approximately 95%.  The variables included were the following:
`
categoricals = [ 'MONTH',  'OP_CARRIER_AIRLINE_ID', 'ORIGIN_AIRPORT_ID', 'DEST_AIRPORT_ID', 
                'SEASON',  'DEPARTURE_Hour_CRS', 'time_of_day_int']
`
`
numerics = [ 'wind_speed_mps_orig', 'ceiling_ht_dim_orig', 'visibility_meters_orig', 'temp_cels_orig', 'dew_pt_orig', 'precip_milimeters_orig',`
`'wind_speed_mps_dest', 'temp_cels_dest',  'precip_milimeters_dest', 'rolling_ninety_day_average', 'OD_delay_pair', 'Coalesced_PgRank_orig']
`

The following table presents the Model Performance metrics for Random Forest with 110 trees and 14 maxDepth (RF 100/14) versus the best alternative configuration found so far (RF 110/16). The new model represents a slight increase in Accuracy (65.7% vs. 64.8%), while maintaining the other metrics at similar levels.  Thus, we propose to continue with this new model for further evaluation.

___Figure 5.14: Reduced Model (19 attributes) vs. Full Model (33 Features)___

| Metric     | RF\_110/14 | RF 110/16 |
| ---------- | ---------- | --------- |
| Accuracy   | 64.8%      | 65.7%     |
| Precision  | 29.7%      | 30.2%     |
| Recall     | 64.8%      | 63.8%     |
| F1\_Score  | 40.8%      | 41.0%     |
| F05\_Score | 33.4%      | 33.8%     |
| F2\_Score  | 52.5%      | 52.2%     |`

The following table presents the Confusion Matrix for the new model. While the model is able to correctly identify delays and no delays in 66.2% and 63.8% of cases respectively, the significantly higher number of on-time flights (82% of data set) reduces the Precision while maintaining a relatively high Recall Score.  

___Figure 5.15: Confusion Matrix for Random Forest 19 features, 110 trees, 16 Max Depth.___

| Label | Pr (0)    | Pr (1)    |
| ----- | --------- | --------- |
| 0     | 3,715,598 | 1,899,150 |
| 1     | 466,926   | 823,411   |

| Label | Pr (0) | Pr (1) |
| ----- | ------ | ------ |
| 0     | 66.2%  | 33.8%  |
| 1     | 36.2%  | 63.8%  |

##### h. Ensemble Algorithm

As a way to improve the `Acuracy` and `F2_score`, we explored `Ensemble` models. First, we trained `Random Forest`(RF), `Gradient Boosted Tree` (GBT), and `Linear Support Vector Classifier` (LSVC) models on data including 2015 to 2019.  Then, we generated predictions for 2019.  

For this implementation we took the predictions as input to a `Logistic Regression` model with Departure Delay (binary variable) as the output. Unfortunately, this approach did not improve performance metrics.  Thus, we suggest continuing exploring Random Forest with 110 trees and 16 maxDepth. 

___Figure 5.16: Ensemble versus based models___

| Metric     | LR\_test | GBT\_test | LSVC test | RF\_test | Ensemble Test |
| ---------- | -------- | --------- | --------- | -------- | ------------- |
| Accuracy   | 59.6%    | 68.3%     | 57.7%     | 65.7%    | 65.2%         |
| Precision  | 27.1%    | 31.7%     | 26.1%     | 30.2%    | 30.0%         |
| Recall     | 68.7%    | 60.3%     | 69.1%     | 63.8%    | 64.8%         |
| F1\_Score  | 38.9%    | 41.5%     | 37.9%     | 41.0%    | 41.0%         |
| F05\_Score | 30.8%    | 35.0%     | 29.8%     | 33.8%    | 33.6%         |
| F2\_Score  | 52.6%    | 51.1%     | 52.0%     | 52.2%    | 52.6%         |`

##### i. Conclusion - Algorithm Evaluation

We first implemented Grid Search using 5% of the data due to the size of the data and time constraints. Thus, we identified optimal parameters for Random Forest, Gradient Boosted Tree, Linear Support Vector Classifier and Logistic Regression. Random Forest was selected for further tuning as it has the best Grid Search Accuracy at 71.3%. However, when Random Forest was trained and tested with the full data, the Accuracy and F2_score was around 64.8% and 52.5% respectively.

We then evaluated Random Forest performance across different variables - such as Season, Month, Time of the Day, Hour of the Day, Airport Group Ranking based on Number of Connections. The breakdown that provided the most insights was analysis by Time of the Day (seven categories), Season (four categories), and Airport Ranking by Connections PageRank (four categories). This analysis suggested that there is potential for implementing independent models by Time of the Day, Season and Airport Group.

We then proceeded to identify the most important variables leveraging Random Forest attribute importance. Engineered features like "OD Delay Pair" and "Time of the Day" were the most important drivers of model performance with a weight of 23.1% and 22.1% respectively. After investing significant effort in implementing PageRank from scratch and creating PageRank features, these features only reached a importance weight of 3.3%. After this analysis, we selected the top 19 attributes (out of 33) that represented 95% of the attribute importance (weight). We also attempted to create features from Community Detection Algorithms but failed due to time and size constraints.
Using the reduced numbers of attributes, we explored several configurations implementing Grid Search with the whole data. Through this process we were able to improve the model performance achieving an `Accuracy`, `Recall`, and `F2_score` of 65.7%, 63.6% and 52.2% respectively.

We implemented an Ensemble model where the predictions of the base models (Random Forest, Gradient Boosted Tree, Linear Support Vector Classifier) were used as input to a Logistic Regression model to predict delay (binary variable). Unfortunately, this approach did not help to improve model performance.


#### VI. Algorithm Implementation

<a href="https://github.com/UCB-w261/w261-sp22-finalproject-team-t14/blob/main/Algorithms%20(Linear%2C%20Logistic%2C%20Multi)%20From%20Scratch.ipynb"> Link to Multi-task Algorithm from Scratch Notebook </a>

For our algorithm implementation, we built the Logistic and Linear regression algorithms from scratch and then combined these two algorithms into a final Multi-Task algorithm.  The two tasks $t_i$ for this Multi-Task algorithm include the prediction of the delay in minutes $t_1$ = Linear Regression) and Binary Classification of Delayed or Not Delayed $t_2$ = Logistic Regression). These two tasks are related because the time delay in minutes is used to determine whether a flight is delayed or not (i.e. 15 + delay minutes the flight is considered to be delayed).

For the purposes of the implementation, we used the following numeric features as the inputs to our model:

- DAY_OF_MONTH
- DAY_OF_WEEK 
- DISTANCE
- wind_speed_mps_orig
- ceiling_ht_dim_orig
- visibility_meters_orig
- temp_cels_orig   
- dew_pt_orig
- atmos_press_orig
- precip_milimeters_orig 
- wind_speed_mps_dest
- ceiling_ht_dim_dest
- visibility_meters_dest
- temp_cels_dest 
- dew_pt_dest 
- atmos_press_dest
- precip_milimeters_dest 
- rolling_ninety_day_average
- Air_Page_Rank_traffic
- OD_delay_pair 
- time_of_day_int


The sections below detail the Pyspark implementation and mathmatical formulas used to create the Linear Regression, Logistic Regression, and Multi-Task algorithms.

##### Linear Regression

The Mean Squared Error (MSE) function was utilized for the Linear regression model as it is a convex function where the minimum is equal to the global minimum.  Overall, the MSE represents the average of the squared residuals, and this can be re-written into an alternate form. See MSE formulas and Python Implementation below.

___MSE Formula___
$$ MSE = \frac{1}{m}\sum_{i=1}^{m}\left(y^{\left(i\right)}-{\hat{y}}^{\left(i\right)}\right)^2 $$

___Alternate MSE Formula___

$$
f(\boldsymbol{\theta}) = \frac{1}{n}\sum_{i=1}^{n}\left[ \boldsymbol{\theta}^T\cdot\mathbf{x}'_i - y_i\right]^2
$$

To find the global minimum of the Mean Squared Error (MSE), we used gradient descent for a linear regression model. The gradient descent update formula updates the weights proportional to the error term. In this way, there are larger updates for predictions with larger errors. For additional details, please see the gradient descent update formula below. 

___Gradient Descent Update Formula___
$$\theta_{j} : = \theta_{j} - \alpha \cdot (\frac{2}{n}\,\sum_{i=1}^{n}\left[ \boldsymbol{\theta}^T\cdot\mathbf{x}'_i - y_i\right] \cdot \mathbf{x}'_i)$$


__Linear Regression Python Implementation__

1 - Normalize the Training Data

``` PYTHON
def normalize(dataRDD, refRDD):
    """
    Scale and center data round mean of each feature.
    Args:
        dataRDD - records are tuples of (features_array, y)
    Returns:
        normedRDD - records are tuples of (features_array, y)
    """
    featureMeans = refRDD.map(lambda x: x[0]).mean()
    featureStdev = np.sqrt(refRDD.map(lambda x: x[0]).variance())
    
    normedRDD = dataRDD.map(lambda x: ((x[0]-featureMeans)/featureStdev,x[1]))
    
    return normedRDD 
```

2 - Create MSE Loss Function 

``` PYTHON
def OLSLoss(dataRDD, W):
    """
    Compute mean squared error.
    Args:
        dataRDD - each record is a tuple of (features_array, y)
        W       - (array) model coefficients with bias at index 0
    """
    #augmentedData = dataRDD.map(lambda x: (np.append([1.0], x[0]), x[1]))
    augmentedData = dataRDD.map(lambda x: (np.append([1.0], x[0]), x[1][1]))
    loss = None
    
    loss = augmentedData.map(lambda x:(np.dot(x[0],W) - x[1])**2).mean()
    
    return loss
```
_Note: we are augmenting the input feature vector to include the bias term._

3 -  Create Gradient Update Function

``` PYTHON
def GDUpdateLR(dataRDD, W, regType = None, learningRate = 0.1, regParam = 0.1):
    """
    Perform one OLS gradient descent step/update.
    Args:
        dataRDD - records are tuples of (features_array, y)
        W       - (array) model coefficients with bias at index 0
    Returns:
        new_model - (array) updated coefficients, bias at index 0
    """
    # add a bias 'feature' of 1 at index 0
    augmentedData = dataRDD.map(lambda x: (np.append([1.0], x[0]), x[1][1])).cache()

    w_broadcast = sc.broadcast(W)
    w = np.append([0.],W[1:])

    if regType =='ridge':
        reg = regParam * 2 * w
    elif regType == 'lasso':
        reg = W * 1
        reg = (reg>0) * 2- 1
        reg[0] = 0
        reg = reg * regParam
    else:
        reg = np.float(0)
    
    grad = augmentedData.map(lambda d: np.dot(d[0], ( np.dot(d[0],W) - d[1] )  )).mean()*2
    
    grad = grad + reg
    new_model = W - (learningRate * grad)
   
    return new_model
```

4 - Create Linear Regression Model Function

``` PYTHON
def OLSRegression(dataLR, nSteps, verbose, modelLR, regType, learningRate, regParam):

    if verbose: print(f"BASELINE:  Loss = {OLSLoss(dataLR,modelLR)}")
    for idx in range(nSteps):
        modelLR = GDUpdateLR(dataLR, modelLR, regType, learningRate, regParam)
        lossLR = OLSLoss(dataLR, modelLR) 
        if verbose:
            print("----------")
            print("STEP: {}".format(str(idx+1)))
            print("Loss: {}".format(str(lossLR)))
            print("Model:{}".format([w for w in modelLR]))
        else:
            print("STEP: {}".format(str(idx+1)))
    
    return modelLR
```

5 - Create RMSE Function to Evaluate the Model

``` PYTHON
def rmse(data, model):
    """
        Compute Root Mean Squared Error (RMSE)
        Args:
            data     - each record is a tuple of (features_array, y)
            model  - (array) model coefficients with bias at index 0
    """
    
    augmentedData = data.map(lambda x: (np.append([1.0], x[0]), x[1][1])).cache()

    
    rmse_result = augmentedData.map(lambda x: (np.dot(x[0],model) - x[1] )**2).mean()
    rmse_result = np.sqrt(rmse_result)
    
    return rmse_result

```

##### Logistic Regression

The MSE cannot be used as the cost function because the logistic regression (sigmoid function) is non-linear which leads to a non-convex function. Thus, to meet the properties of convexity the Log Loss function is used in place of MSE. The Log Loss function evaluates binary classification performance for probability values between 0 and 1 by calculating the negative of the log likelihood function.  See formulas below and Logistic Regression Python Implementation for more details.

___Logistic (Sigmoid) Function___

When plotted the sigmoid function creates an S-shaped curve on the graph with Y-axis values that range from 0 to 1. After applying the sigmoid function, a value is converted into a value between 0 and 1 which then can be interpreted as the a probability. In our implementation, if the outcome of the sigmoid function is greater than .5 we predict that the flight will be delayed (1). If the outcome is less than or equal to .5 we predict that the flight will be on-time (0).

$$g(z)=\frac{1}{1+e^{(-z)}}$$

___Logistic Regression Loss Function (Log Loss Function)___

$$J\left(\theta\right)=-\frac{1}{m}[\sum_{i=1}^{m}{y^{\left(i\right)}log{h_\theta\left(x^{\left(i\right)}\right)}}+\left(1-y^{\left(i\right)}\right)log{\left(1-h_\theta\left(x^{\left(i\right)}\right)\right)}]$$

where ___m___ is the number of examples


The gradient update formula resembles the gradient update for the linear regression formula, however, the $h_\theta(x)$ is the logistic regression hypothesis for the logistic regression. We can see below that the Logistic regression hypothesis contains the sigmoid function.

___Logistic Regression Hypothesis___

$$h_\theta\left(x\right)=\frac{1}{1+e^{-\theta^Tx}}$$

___Gradient Descent Update Formula___

$$ 
\theta_j=\theta_j-\alpha\sum_{i=1}^{m}{(h_\theta\left(x^{\left(i\right)}\right)}-y^{\left(i\right)})x_j^{\left(i\right)}
$$


__Logistic Regression Python Implementation__

1 - Normalize the Training Data

``` PYTHON
def normalize(dataRDD, refRDD):
    """
    Scale and center data round mean of each feature.
    Args:
        dataRDD - records are tuples of (features_array, y)
    Returns:
        normedRDD - records are tuples of (features_array, y)
    """
    featureMeans = refRDD.map(lambda x: x[0]).mean()
    featureStdev = np.sqrt(refRDD.map(lambda x: x[0]).variance())
    
    normedRDD = dataRDD.map(lambda x: ((x[0]-featureMeans)/featureStdev,x[1]))
    
    return normedRDD 
```

2 - Create Sigmoid and Log Loss Functions 

``` PYTHON
# helper function: sigmoid function
def sigmoid(z):
    return 1.0/(1+np.exp(-z))

# Log Loss Function
def LogLoss(dataRDD, W):
    """
    Compute the log loss.
    Args:
        dataRDD - each record is a tuple of (features_array, y)
        W       - (array) model coefficients with bias at index 0
    """
    augmentedData = dataRDD.map(lambda x: (np.append([1.0], x[0]), x[1][0]))
    #  Log Loss =                                    prediction 1                       +            prediction  0
    # Log Loss =                            y *    log (sigmoid (dot(X,weights)))      + (1-y) * log (1- sigmoid(dot(X,weights)))   . mean
    loss =  augmentedData.map(lambda X_y: ((X_y[1] * np.log(sigmoid(np.dot(X_y[0], W))))+ ((1-X_y[1])*np.log(1-sigmoid(np.dot(X_y[0], W)))))).mean()
    loss = -loss
    return loss
```
_Note: we are augmenting the input feature vector to include the bias term._

3 -  Create Gradient Update Function

``` PYTHON
def GDUpdate(dataRDD, W, regType = None, learningRate = 0.1, regParam = 0.1):
    """
    Perform one OLS gradient descent step/update.
    Args:
        dataRDD - records are tuples of (features_array, y)
        W       - (array) model coefficients with bias at index 0
    Returns:
        new_model - (array) updated coefficients, bias at index 0
    """
    # add a bias 'feature' of 1 at index 0
    augmentedData = dataRDD.map(lambda x: (np.append([1.0], x[0]), x[1][0])).cache()
    w_broadcast = sc.broadcast(W)
    w = np.append([0.],W[1:])

    if regType =='ridge':
        reg = regParam * 2 * w
    elif regType == 'lasso':
        #reg = 1 * np.sign(w)
        reg = W * 1
#         reg = (reg>0).astype(int) * 2- 1
        reg = (reg>0) * 2- 1
        reg[0] = 0
        reg = reg * regParam
    else:
        reg = np.float(0)
    
    #                               X               .        (sigmoid(X   .    Theta )     - y)          / m 
    grad = augmentedData.map(lambda x: np.dot(x[0], (sigmoid(np.dot(x[0],w_broadcast.value))-x[1]))).mean()
    #
    grad = grad + reg
    new_model = w_broadcast.value - learningRate * grad
  
    #return grad
    return new_model
```

4 - Create Logistic Regression Model Function

``` PYTHON
def LogRegression(data, nSteps, verbose, model, regType, learningRate, regParam):

    if verbose: print(f"BASELINE:  Loss = {LogLoss(data,model)}")
    for idx in range(nSteps):
        model = GDUpdate(data, model, regType, learningRate, regParam)
        loss = LogLoss(data, model) 
        if verbose:
            print("----------")
            print("STEP: {}".format(str(idx+1)))
            print("Loss: {}".format(str(loss)))
            print("Model:{}".format([w for w in model]))
        else:
            print("STEP: {}".format(str(idx+1)))
    
    return model
```

5 - Create `logmetrics` Function to Evaluate the Model on Accuracy, Recall, Precision, F1 Score, F-05, Score, and F2 Score

``` PYTHON
def logrmetrics(metrics):
    dfmetric = {}
    recall = metrics.recall(1.0)
    precision = metrics.precision(1.0)
    dfmetric["accuracy"] = metrics.accuracy
    dfmetric["recall"] = recall
    dfmetric["precision"] = precision
    if (recall + precision) != 0:
        dfmetric["f1_score"] = 2*(recall * precision) / (recall + precision)
    else:
        dfmetric["f1_score"] = "na"
    beta = 0.5
    if ((beta**2 * precision) + recall) != 0:
        dfmetric["f05_score"] = (1+beta**2)*(recall * precision) / ((beta**2 * precision) + recall)
    else:
        dfmetric["f05_score"] = "na"
    beta = 2
    if ((beta**2 *precision) + recall) !=0:
        dfmetric["f2_score"] = (1+beta**2)*(recall * precision) / ((beta**2 *precision) + recall) 
    else:
        dfmetric["f2_score"] = "na"
    
    return dfmetric

```

##### Multi-Task Algorithm

Using the multi-task algorithm, the logistic and linear regressions loss functions are weighted and then summed into a single multi-task loss function (see formula below). This deviates from single-task (supervised) machine learning algorithm which seeks to minimize a single loss function with respect to the parameters. We applied normalization to $t_2$ (linear regression) labels because these task labels are not on the same scale and the multi-task loss would be skewed to these labels that have a greater magnitude.

___Simplified Multi-Task Loss Function___
$$ Multi-Task\ Loss=\left(1-\beta\right)LogLoss+\left(\beta\right)MSE $$

___Multi-Task Loss Function___

$$ Multi-Task\ Loss =(1-\beta)\left(-\frac{1}{m}\sum_{i=1}^{m}{y^{\left(i\right)}log{h_\theta\left(x^{\left(i\right)}\right)}}+\left(1-y\left(i\right)\right)log{\left(1-h_\theta\left(x^{\left(i\right)}\right)\right)}\right)+\left(\beta\right)\frac{1}{m}\sum_{i=1}^{m}\left(y^{\left(i\right)}-{\hat{y}}^{\left(i\right)}\right)^2$$
_where __β__=weight of each loss function_


Like the single-task (Logistic and Linear regression) algorithms, we used a gradient descent update which was calculated by taking the partial derivative of the combined loss function with respect to the feature vector (X). Finally, we apply regularization by adding in the regularization penalties to the gradient prior to multiplying by the learning rate. 

___Multi-Task Gradient___

$$Gradient = (1-\beta)\ \ast\ \sum_{i=1}^{m}{{(h}_\theta\left(x^{\left(i\right)})-y^{\left(i\right)}\right)x_j^{\left(i\right)}\ +\ }\left(\beta\right)\frac{2}{m}\sum_{i=1}^{m}{{(\left[\theta^T\cdot x_i^\prime-y_i\right]}^\ast x_i^\prime)}$$

___Solving for Multi-Task Gradient___

$$\frac{\partial}{\partial x}(1-\beta)\left(-\frac{1}{m}\sum_{i=1}^{m}{y^{\left(i\right)}log{h_\theta\left(x^{\left(i\right)}\right)}}+\left(1-y\left(i\right)\right)log{\left(1-h_\theta\left(x^{\left(i\right)}\right)\right)}\right)+\left(\beta\right)\frac{1}{m}\sum_{i=1}^{m}\left(y^{\left(i\right)}-{\hat{y}}^{\left(i\right)}\right)^2$$



__Multi-Task Algorithm Python Implementation__

1 - Normalize the Training Data: _We use a different Normalize function for the Linear Regression that also normalizes the labels for the combined loss function_

``` PYTHON
# logistic regression normalize
def normalize(dataRDD, refRDD):
    """
    Scale and center data round mean of each feature.
    Args:
        dataRDD - records are tuples of (features_array, y)
    Returns:
        normedRDD - records are tuples of (features_array, y)
    """
    featureMeans = refRDD.map(lambda x: x[0]).mean()
    featureStdev = np.sqrt(refRDD.map(lambda x: x[0]).variance())
    
    normedRDD = dataRDD.map(lambda x: ((x[0]-featureMeans)/featureStdev,x[1]))
    
    return normedRDD 
  
# linear regression normalize 

def normalizeLR(dataRDD, refRDD):
    """
    Scale and center data round mean of each feature.
    Args:
        dataRDD - records are tuples of (features_array, y)
    Returns:
        normedRDD - records are tuples of (features_array, y)
    """
    featureMeans = refRDD.map(lambda x: x[0]).mean()
    featureStdev = np.sqrt(refRDD.map(lambda x: x[0]).variance())
    
    yMean = refRDD.map(lambda x: x[1][1]).mean()
    yStdev = np.sqrt(refRDD.map(lambda x: x[1][1]).variance())
    
    normedRDD = dataRDD.map(lambda x: ( (x[0]-featureMeans) / featureStdev, (x[1][0], (x[1][1]-yMean)/yStdev))  )
    
    return normedRDD

```

2 - Use Sigmoid and created a combined MSE and Log Loss Functions. _This includes a parameter for beta which is the weight value to be applied to each loss._

``` PYTHON
# helper function: sigmoid function
def sigmoid(z):
    return 1.0/(1+np.exp(-z))

# Multi-Task Loss Function
def MTLoss(data, modelLR, modellogr, beta):
    """
    Compute multi-task loss function
    Args:
        data     - each record is a tuple of (features_array, y)
        modelLR  - (array) Linear Regression model coefficients with bias at index 0
        modellogr - (array) Logistic Regression model coefficients with bias
        beta - (numeric) weight for the loss function of each
    """
    augmentedDF = data.map(lambda x: (np.append([1.0], x[0]), x[1]))
    lossLR = None
    
    multi_loss = augmentedDF.map(lambda x:( (np.dot(x[0],modelLR) - x[1][1])**2, ((x[1][0] * np.log(sigmoid(np.dot(x[0], modellogr))))+ ((1-x[1][0])*np.log(1-sigmoid(np.dot(x[0], modellogr)))))         ))


    
    lossLogR = multi_loss.map(lambda x: x[1]).mean()
    lossLogR = -lossLogR
    lossLR = multi_loss.map(lambda x: x[0]).mean()
    
    # Multi-task loss
    MTLossVal = lossLR*beta + (1-beta)*lossLogR
       
    return MTLossVal
```
_Note: we are augmenting the input feature vector to include the bias term._

3 -  Create Gradient Update Function for Multi-Task Loss Function

``` PYTHON
def GDMTUpdateLR(data, modelLR, modellogr, beta, LRregType, LRregParam,logregType, logregParam, learningRate = 0.1):
    #helper function 
    def get_reg( W, regType, regParam):
        w = np.append([0.],W[1:])
        if regType =='ridge':
            reg = regParam * 2 * w
        elif regType == 'lasso':
            reg = W * 1
            reg = (reg>0) * 2- 1
            reg[0] = 0
            reg = reg * regParam
        else:
            reg = np.float(0)
        return reg
    
    
    modellogr_broadcast = sc.broadcast(modellogr)
    modelLR_broadcast = sc.broadcast(modelLR)

    #UPDATE MODEL FOR LINEAR REGRESSION
    # add a bias 'feature' of 1 at index 0
    augmentedDF =data.map(lambda x: (np.append([1.0], x[0]), x[1])).cache()
    
    #gradLR = augmentedLR.map(lambda d: np.dot(d[0], ( np.dot(d[0], modelLR) - d[1] )  )).mean()*2
    grads = augmentedDF.map(lambda d: (np.dot(d[0], ( np.dot(d[0], modelLR_broadcast.value) - d[1][1] )  )  ,np.dot(d[0], (sigmoid(np.dot(d[0],modellogr_broadcast.value))-d[1][0])))).cache()
    
    # Get regularization for each model
    lrReg = get_reg(modelLR_broadcast.value, LRregType, LRregParam)
    logReg = get_reg(modellogr_broadcast.value, logregType, logregParam)
    
    # add regularization to the gradients
    gradLR = grads.map(lambda x: x[0]).mean()*2 + lrReg
    grad_LogR = grads.map(lambda x: x[1]).mean() + logReg
    
    new_model_LR = modelLR - (learningRate * gradLR * beta)
        
    new_model_logR = modellogr_broadcast.value - (learningRate * grad_LogR * (1- beta))
    
    return new_model_LR, new_model_logR
```

4 - Create Multi-Task Model Function

``` PYTHON
def MTRegression(data, nSteps, verbose, modelLR, modellogr, beta, LRregType, LRregParam,logregType, logregParam, learningRate):

    if verbose: print(f"BASELINE:  Loss = {MTLoss(data, modelLR, modellogr, beta)}")
    for idx in range(nSteps):
        modelLR, modellogr = GDMTUpdateLR(data, modelLR, modellogr, beta, LRregType, LRregParam,logregType, logregParam, learningRate)
        lossMT = MTLoss(data, modelLR, modellogr, beta) 
        if verbose:
            print("----------")
            print("STEP: {}".format(str(idx+1)))
            print("Loss: {}".format(str(lossMT)))
            print("Model LR:{}".format([w for w in modelLR]))
            print("Model LogR:{}".format([w for w in modellogr]))
        else:
            print("STEP: {}".format(str(idx+1)))
    
    return modelLR, modellogr
```

5 - Use logmetrics Function to Evaluate the Logistic Regression model and RMSE to evaluate the Linear Regression Model

``` PYTHON
def logrmetrics(metrics):
    dfmetric = {}
    recall = metrics.recall(1.0)
    precision = metrics.precision(1.0)
    dfmetric["accuracy"] = metrics.accuracy
    dfmetric["recall"] = recall
    dfmetric["precision"] = precision
    if (recall + precision) != 0:
        dfmetric["f1_score"] = 2*(recall * precision) / (recall + precision)
    else:
        dfmetric["f1_score"] = "na"
    beta = 0.5
    if ((beta**2 * precision) + recall) != 0:
        dfmetric["f05_score"] = (1+beta**2)*(recall * precision) / ((beta**2 * precision) + recall)
    else:
        dfmetric["f05_score"] = "na"
    beta = 2
    if ((beta**2 *precision) + recall) !=0:
        dfmetric["f2_score"] = (1+beta**2)*(recall * precision) / ((beta**2 *precision) + recall) 
    else:
        dfmetric["f2_score"] = "na"
    
    return dfmetric
  
  def rmse(data, model):
    """
        Compute Root Mean Squared Error (RMSE)
        Args:
            data     - each record is a tuple of (features_array, y)
            model  - (array) model coefficients with bias at index 0
    """
    
    augmentedData = data.map(lambda x: (np.append([1.0], x[0]), x[1][1])).cache()

    
    rmse_result = augmentedData.map(lambda x: (np.dot(x[0],model) - x[1] )**2).mean()
    rmse_result = np.sqrt(rmse_result)
    
    return rmse_result

```

##### Results

To validate that our From Scratch algorithm implemenations we compared our model results against the PySpark MLlib implemention. Prior to running the algorithm we selected relevant numeric features and then normalized them by subtracting the mean and dividing by the standard deviation of the training data. Specifically, we tested our algorithms using 2.5% of records randomly selected from the dataset and split this into a train and test dataset. We ran our From Scratch implementations for 500 iterations to get the model estimates. As exhibited in Figure 6.1, the From Scratch models performed similarly to their MLlib counterparts for all Linear Regression models including those with Ridge and Lasso Regularization. Additionally, the coefficients for our From Scratch models were approximately equal to the coefficients generated from the MLlib.

___Figure 6.1: Pyspark versus From Scratch Implementation Validation___

| Metrics     | LR Scratch | PS LogR | Rige Scratch | PS Ridge | Lasso Scratch | PS Lasso |
| ----------- | ---------- | ------- | ------------ | -------- | ------------- | -------- |
| Accuracy    | 81.1%      | 81.1%   | 81.2%        | 81.1%    | 81.3%         | 81.1%    |
| Precision   | 1.6%       | 1.7%    | 0.2%         | 0.5%     | 0.0%          | 0.5%     |
| Recall      | 37.7%      | 37.8%   | 24.2%        | 29.3%    | 0.0%          | 29.3%    |
| Specificity | 3.1%       | 3.2%    | 0.4%         | 1.0%     | na            | 1.0%     |
| F1 Score    | 7.0%       | 7.2%    | 1.1%         | 2.5%     | na            | 2.5%     |
| F05 Score   | 2.0%       | 2.1%    | 0.3%         | 0.7%     | na            | 0.7%     |`

In Figure 6.1, the Lasso shows slightly performance metrics but the data coefficients were getting closer to the MLlib cofficients. Additionally, the differences in the Ridge regressions suggest that the From Scracth model may require more iterations to converge.

___Figure 6.2 Linear Regression Pyspark vs From Scratch___

| RMSE  | LN Scratch | PS LinR | Ridge Scratch | PS Ridge | Lasso Scratch | PS Lasso |
| ----- | ---------- | ------- | ------------- | -------- | ------------- | -------- |
| Train | 41.149     | 41.149  | 41.149        | 42.872   | 41.151        | 42.89    |
| Test  | 48.805     | 48.805  | 48.805        | 50.546   | 48.811        | 50.563   |`

Figure 6.2, further illustrates how we were able to acheive similar results accross the board with the Linear Regression from Scratch implementation versus MLlib. Given the 500 iterations for the From Scratch implementation, we have slight differences in performance with the Ridge and Lasso regression.

___Figure 6.3 Multi-Task Algorithm From Scratch___

| Metrics     | Ordinary | Lasso | Ridge |
| ----------- | -------- | ----- | ----- |
| Accuracy    | 81.1%    | 81.3% | 81.2% |
| Precision   | 1.6%     | 0.0%  | 0.2%  |
| Recall      | 37.7%    | 0.0%  | 24.2% |
| Specificity | 3.1%     | na    | 0.4%  |
| F1 Score    | 7.0%     | na    | 1.1%  |
| F05 Score   | 2.0%     | na    | 0.3%  |`

As shown in Figure 6.3 above, the Lasso Logistic model produced from the Multi-task algorithm performs slighly better in Accuracy than the Ridge and Ordinary implementation, but performs poorly in Precision and Recall. 

_Note: this is based on 2.5% of the test data, we will use the complete test data later on._

___Figure 6.4 Multi-Task Algorithm From Scratch___

| RMSE  | Ordinary | Lasso  | Ridge  |
| ----- | -------- | ------ | ------ |
| Test  | 50.567   | 50.591 | 50.570 |
| Train | 42.896   | 42.916 | 42.899 |`

Using 2.5% of the Test data and a beta value of .4, the Multi-Task algorithm has a slightly lower RMSE for the Ordinary implementation than when regularization is applied.

In the final iteration, we trained the Ordinary (No Regularization) From Scratch implementations on 2.5% of the training data from 2015 to 2018 and then tested the models on the entire test dataset (2019 data). Figure 6.5 compares the performance of these models in terms of RMSE and  Accuracy. Interestingly, Figure 6.5 shows that the Multi-Task algorithm performs better than the Linear Regression in terms of RMSE on the Test data but worse than the Linear Regression on the 2.5% of training data. Additionally, the Multi-Task model performs similarly in Accuracy to the standard implementation of the Logistic regression (58.69% vs. 58.70%).

___Figure 6.5 From Scratch Algorithm Comparison___

| Dataset          | Multi - Linear (RMSE) | Linear (RMSE)     |
| ---------------- | -------------- | ----------- |
| Test (Full 2019) | 49.820668         | 52.296001 |
| Train (2.5%)     | 70.557089    | 62.080181 |


___Figure 6.6 From Scratch Logistic Algorithm Comparison___

|    Dataset       | Multi - Logistic (Accuracy) | Logistic (Accuracy) |
| ---------------- | ---------------- | -------- |
| Test (Full 2019) | 0.586973         | 0.587091 |

The multi-task algorithm did not perform as well as the log regression implemented in the previous section (Algorithm Exploration) because we only selected numeric features and only trained it on 2.5% of the data.

#### VII. Conclusions

We hypothesized that weather, temporal, and late aircraft related factors were most impactful to flight delays. 
From our Random Forest Model, OD-Delay pair and time-of-day were the two most important features. OD-Delay pair captures prior conditions related to multiple causes of delays. Time-of-day captures the temporal and cumulative effects of delays as the day progresses. Weather-related features as a collective accounted for 18% of the total feature importance. However, what differed from our expectations was that individual weather features were not as significant as OD-Delay Pair and Time-of-day features. In addition, some features such as weekday versus weekend, quarter and flight distance were the least important features.

For scalability, we utilized the MLlib implementation of Gradient Boosted Trees, Random Forest, and Logistic Regression. The MLlib designs these Machine Learning algorithms to be parallelizable and works natively within Spark. The key benefits of using MLlib are in-memory process, fault-tolerance, speed, and ease of programming. Therefore, since we leveraged simple algorithms that can be parallelized by pyspark and we do not foresee major scalability issues. Even though, Random Forest is a parallelized algorithm, running this with the best grid-search parameters was not viable and we had to select and reduce the number of features based on importance. Our best model was ensemble learning which employs multiple models and hence it may be more computationally intensive and time-consuming. Lastly, we attempted to work on engineered features with Community Detection Algorithms but were not successful due to long run time and lack of memory space. 

In terms of evaluating our model performance against our research objective of predicting delayed versus non-delayed flights. We selected the majority class of non-delays of 82% as the baseline accuracy which gives us an undefined F1 and F2-Score initially. Our model performance results show that the Random Forest model (110 trees and 16 maxDepth) offers an `Accuracy`, `Recall`, and `F2_score` of 65.7%, 63.6% and 52.2% respectively. Using our best performing model the proportion of true positives (correctly predicted delays) to actual delays in 2019 is 63.8%  which represents an estimated `$4.1` billion cost to passengers (out of `$6.4` billion). Thus, we expect that our model can save passengers an estimated `~$4` billion annually if they were able to make alternative arrangements. 

We also implemented the multi-task algorithm from scratch in PySpark. In this implementation we achieved 58.7% accuracy, which demonstrates that there could be a benefit to combining the loss functions of Linear (MSE) and Logistic (Log Loss) into multi-task loss function. Our hypothesis is that since the loss functions could be trained in parallel, we could train both the Multi-Task Logistic and Multi-Task Linear Regressions faster than the independent regression implementations. Therefore, there is an opportunity to improve the scalability of the algorithm by parallelizing the multi-task algorithm further. 

For future exploration purposes, an opportunity exists to improve our model performance by creating custom models for certain seasons and time-of-day as mentioned in the above sections of our paper. Given more time, we would have also have explored more sophisticated models like XGBoost and Neural Networks for feasibility in a big data setting. Also, we would have enhanced the grid-search so that it optimized not just by Accuracy but also by F2-Score.

#### VIII. Application of Course Concepts 

**Distributed processing with Spark:** Apache Spark is a general-purpose distributed data processing engine. Spark is faster than Hadoop because Hadoop writes intermediate results to disk whereas Spark tries to keep intermediate results in memory. Moreover, Spark offers lazy evaluation of operations and optimizes them just before the final result. Spark optimizes network communication by exploiting the laziness (on transformation) and eagerness (on action). 

**Tradeoffs:** A good machine learning model is all about making the right tradeoffs. For example, balancing between overfitting and underfitting models, or balancing between bias and variance. The Random Forest employs techniques to maintain low variance that is characteristic of a single Decision Tree. Random Forest does this by averaging together a number of very weakly correlated trees. Hyperparameters like num_trees and max_depth are among the techniques useful in reducing this correlation between trees, but they often come at the cost of some increase in bias, since each tree now has less data to work with. By tuning the value of the tree’s depth, we seek to balance the bias-variance tradeoff. If the depth is too large, our tree will grow too deep and overfit the training data. Conversely, if the depth is too small, our tree will not grow deep enough and will underfit the training data. Therefore, we need to use cross validation to identify the value of depth that balances this tradeoff on our data.

__Scalability / Time complexity / I/O vs Memory__

In some cases, the data in Spark can be unevenly distributed across partitions, this can prevent the full utilization of parallel processing and make tasks more computationally expensive. One technique we utilized was the repartitioning of the flights dataset to achieve the ideal partition size of 128 mb per partition. Additionally, we applied a `partitionby()` to the `station_id` column on the weather table which is a primary key that we join on. By making these two changes we were able to reduce the join time of the weather and flights dataset from 3.2 hours to 1.9 hours.

```
#Optimize number of partitions for dataframe

# First check number of spark partitions = 200
df_airlines.rdd.getNumPartitions()

# Find the size of the dataset

df_airlines.cache().foreach(lambda x: x) 
catalyst_plan = df_airlines._jdf.queryExecution().logical() 
spark._jsparkSession.sessionState().executePlan(catalyst_plan).optimizedPlan().stats().sizeInBytes()
# Revised Dataset
# 6440 mb/ 200 partitions = 33 mb per partition but the ideal is 128mb per partition
# 6,440,582,293

#change number of partitions such that we can achieve 128mb per partition
# 6440 mb / 128mb = 51
df_airlines_new = df_airlines.repartition(51)

#Check to see that number of partitions for dataframe is now 51
df_airlines_new.rdd.getNumPartitions()


```
**Spark Performance Boosting:** Here we will point out some techniques we used to boost our SQL join performances:

- Broadcast: Joining two tables is one of the main operations in Spark. It mostly requires shuffle which has a high cost due to data movement between nodes. If one of the tables is small enough, any shuffle operation may not be required. By broadcasting the small table to each node in the cluster, shuffle can be simply avoided.
- `paritionBy()`: This is a Pyspark function that enables us to partition a large Spark Dataframe based on a column. Partitionby can reduce the amount of shuffling required by a join and improve the overall runtime.
- Cache: The execution memory and storage memory share a unified region. The more unnecessary caching, the more chance it to spill onto the disk which is a performance hit. In this way, recomputation may be faster than the price paid by the increased memory pressure. Several storage levels are available in Spark, it might be set accordingly in terms of the serialization, memory, and data size factors. If a data frame will be used in the following steps again and again iteratively, it would be rational to cache it at the beginning to avoid repetitive transformation loads. That is an ideal case of using cache.
- Unpersist: Unpersist is an exact opposite of Caching. If the data in the cache, if you do not need it for the rest of the code.
- UDF: Avoid using UDFs unnecessarily is a good practice while developing in Pyspark. Built-in Spark SQL functions mostly supply the requirements. It is important to rethink before using UDFs in Pyspark. UDF,comes together with a very high cost in Pyspark. They operate one row at a time and thus suffer from high serialization and invocation overhead.

__Gradient Descent - Convex Optimization:__ Gradient Descent is an optimization algorithm used to find the minimum of a loss function. For our Linear, Logistic, and Multi-Task From Scratch Algorithms we selected convex loss functions (Log Loss and MSE). By using convex loss functions, the gradient descent will converge at the global minimum and not a local minimum. 


**One Hot Encoding / feature selection:** We chose to use One Hot Encoding (OHE) for categorical variables. We used Feature importance to select the features that accounted for 95% of the total feature total importance leveraging Random Forest Algorithm. We normalized numerical features so that they were on the same scale. By normalizing the features, they are equally weighted which helped the ML algorithms to detect the most important feature.

#### IX. References 

● (BTS, 2022), Transtats, Bureau of Transportation Statistics,
https://www.transtats.bts.gov/Data_Elements.aspx?Data=2

● Weather related delays in summer months, https://www.faa.gov/nextgen/programs/weather/faq/

● (Carvalho et al, 2020), On the relevance of data science for flight delay research: a systematic review, Transport Reviews.  https://arxiv.org/abs/1703.06118

● (Govinda et al, 2017), Cloud-based flight delay prediction using logistic regression, 2017 International Conference on Intelligent Sustainable Systems (ICISS). https://ieeexplore.ieee.org/document/8389254

● (Wong et al, 2012), A survival model for flight delay propagation. Journal of Air Transport Management, 23(Supplement C):5–11, Aug. 2012. ISSN 0969-6997. https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.825.2315&rep=rep1&type=pdf



● (Bao et al, 2021), Graph to sequence learning with attention mechanism for network-wide multi-step-ahead flight delay prediction, https://www.sciencedirect.com/science/article/pii/S0968090X21003296

● (Chakrabarty, 2019), A Data Mining Approach to Flight Arrival Delay Prediction for American Airlines, https://arxiv.org/abs/1903.06740

● (Rebollo et al, 2014), Characterization and prediction of air traffic delays, https://web.mit.edu/hamsa/www/pubs/RebolloBalakrishnanTRC2014.pdf